In [3]:
# Create DataFrame
import re
import numpy as np
import pandas as pd
# Visualize
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
# Connect Portgresat
import psycopg2
# Connect MySQL
import mysql.connector
import pyodbc
# Connect Google Sheet
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
# Time
from datetime import datetime, timedelta
import datetime
import time
import schedule
import calendar
# System
import os
import sys
import pickle
import warnings
warnings.filterwarnings('ignore') # os.chdir(r'C:\Connection')
# Format Mail
from pretty_html_table import build_table
from bs4 import BeautifulSoup
#MAil
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
#from email.message import EmailMessage

C:\Users\antv\AppData\Local\Temp\ipykernel_19008\1125222544.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Test Data

In [7]:
#### Lấy data postgresql
conn_post = psycopg2.connect(host="172.30.105.112",database="y4a_datamart",user="y4a_dl_vylm",password="ACB!asdlkj123")
#### Data BI (Quang Ù)
conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")
#### Data BI (Quang Ù) -- Spend Daily
conn_post_2 = psycopg2.connect(host="172.29.15.98",database="bi_sem", user="da_report",password="Inno202$")
#### Connet Google Sheet get OKR
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(r"G:\Shared drives\Deployment Data Template\Power BI\Python Code\2.Daily\email-sending-via-python-fd13e6126e20.json", scope)
client = gspread.authorize(creds)
service = build('sheets', 'v4', credentials=creds)
sheet_target_OKR = '1WtDK3oV0wJGgloKW_xCZ2PhyLGRQ72GJYzwEDWaMQB4'
#https://docs.google.com/spreadsheets/d/1WtDK3oV0wJGgloKW_xCZ2PhyLGRQ72GJYzwEDWaMQB4/edit#gid=786949489
range_name_OKR = 'Copy of Target OKR Jan 24!B3:K'
result_OKR = service.spreadsheets().values().get(
spreadsheetId=sheet_target_OKR, range=range_name_OKR).execute()
values_OKR = result_OKR.get('values', [])
df_OKR_GG = pd.DataFrame(result_OKR['values'][1:], columns=result_OKR['values'][0])
#### Lấy list Asin, SKU
list_asin_sku = pd.read_sql_query(f"""
    select asin, sku from public.launching_plan where asin is not null and sku is not null """,conn_post_1)
#### List_asin
list_asin = tuple(list_asin_sku['asin'])
#### List_sku
list_sku = tuple(list_asin_sku['sku'])
#### Lấy ngày data
now = datetime.datetime.now()
day = datetime.timedelta(days=1)
data_date = now - day
data_time = data_date.strftime("%Y-%m-%d")
formatted_time = data_date.strftime("ngày %d tháng %m")
formatted_time_daily = data_date.strftime("%d-%b")
Subject_time = data_date.strftime("%d %b").upper()

####Lấy ngày có data real sales mới nhất
df_max_date_real_sales = pd.read_sql(f'''
    select MAX(log_date) as MAX_DAY from y4a_sop_sell_out.sale_performance_by_channel_dpl
    where channel in ('AVC WH','AVC DI') and label_datetime = 'daily' and asin in {list_asin} 
    ''', con=conn_post)
#and log_date <= '2023-12-07'
df_max_date = df_max_date_real_sales.iloc[0]['max_day']
max_data_time = df_max_date.strftime("%Y-%m-%d")
max_data_time_mysql = df_max_date.strftime("%d-%m-%Y")
    
####Lấy data ngày có real sales mới nhất
df_real_sales_MTD = pd.read_sql(f'''
    select asin as ASIN, 
        SUM(CASE WHEN log_date <= '{max_data_time}' 
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
            THEN ordered_units ELSE 0 END) as SELLOUTMTD,
        SUM(CASE WHEN log_date <= '{max_data_time}' 
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
            THEN ordered_gmv ELSE 0 END) as GMVOUTMTD    
    from y4a_sop_sell_out.sale_performance_by_channel_dpl
    where ASIN in {list_asin}
    and country = 'USA' and channel not like 'WM%' and channel not like 'WF%' and channel not like 'ASC%'
    group by ASIN;
    ''', con=conn_post) 
df_real_sales_MTD.columns = df_real_sales_MTD.columns.str.upper()   

#### Lấy data sell in
df_sell_in = pd.read_sql(f'''
SELECT Asin,  
    SUM(CASE WHEN date <= '{data_time}'     
        AND DATE_TRUNC('month', date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
        THEN ordered_unit ELSE 0 END) AS SellInMTD, 
    SUM(CASE WHEN date = '{data_time}'and channel in ('AVC DS','AVC WH','AVC DI') THEN ordered_unit
        ELSE 0 END) AS SellIn
FROM y4a_sop_sell_in.sell_in_perfomance_ordered_date_by_location
where asin in {list_asin}
GROUP BY Asin; ''', con=conn_post)
df_sell_in.columns = df_sell_in.columns.str.upper()
    
#### Lấy data sell in detail
df_detail_sell_in = pd.read_sql(f'''
SELECT Asin,  
    SUM(CASE WHEN date = '{data_time}' and channel = 'AVC DS' THEN ordered_unit ELSE 0 END) AS AVCDS,
    SUM(CASE WHEN date = '{data_time}' and channel = 'AVC WH' THEN ordered_unit ELSE 0  END) AS AVCWH,
    SUM(CASE WHEN date = '{data_time}' and channel = 'AVC DI' THEN ordered_unit ELSE 0  END) AS AVCDI,
    SUM(CASE WHEN date = '{data_time}' and channel in ('AVC DS','AVC WH','AVC DI') THEN ordered_unit ELSE 0 END) AS AllChannel
FROM y4a_sop_sell_in.sell_in_perfomance_ordered_date_by_location
where asin in {list_asin}
GROUP BY Asin; ''', con=conn_post)
df_detail_sell_in.columns = df_detail_sell_in.columns.str.upper()
    
#### Lấy data WM bảng Postgre
df_WM = pd.read_sql_query(f'''
with T1 as (
    select DATE_TRUNC('day', Cover_order_date) as log_date, left(item_sku,4) as sku, Count(order_line_id) as ordered_units, 
        SUM(product_charge_amt) as ordered_gmv from (
            SELECT order_date, order_date - INTERVAL '7 hours' as Cover_order_date, order_line_id, item_sku, product_charge_amt, 
            item_product_name FROM y4a_inno.tb_dpl_wm_dsv_order_rev_dtl
            where left(item_sku,4) in {list_sku}
            order by order_date desc
    ) ABC
    where DATE_TRUNC('month', Cover_order_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
    AND DATE_TRUNC('year', Cover_order_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
    group by DATE_TRUNC('day', Cover_order_date), sku
    order by log_date desc
), T2 as (
    select DATE_TRUNC('day', Cover_order_date) as log_date, left(sku,4) as sku, sum(qty) as ordered_units, 
        sum(prd_charge) from (
            select order_date - INTERVAL '7 hours' as Cover_order_date, sku, qty, prd_charge 
            from y4a_inno.tb_dpl_wm_wsc_ord_dtl_by_sku
            where left(sku,4) in {list_sku}
            and sku_status <> 'Cancelled'
            order by order_date desc
    ) ABCD
    Where DATE_TRUNC('month', Cover_order_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
    AND DATE_TRUNC('year', Cover_order_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
    group by log_date, sku
    order by log_date
), T3 as ( 
SELECT * FROM T1
UNION
SELECT * FROM T2
) 
select sku, 
    SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') = '{data_time}' THEN ordered_units ELSE 0 END) as SELLWM,
    SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_WM,
    SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
        THEN ordered_units ELSE 0 END) as WM_MTD,
    SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
    THEN ordered_gmv ELSE 0 END) as GMV_WM_MTD
from T3
group by sku ''', conn_post)
df_WM.columns = df_WM.columns.str.upper()
#### Lấy data Wayfair
df_WF = pd.read_sql(f'''
    select sku, 
    SUM(CASE WHEN log_date = '{data_time}' THEN ordered_units ELSE 0 END) as SELLWF,
    SUM(CASE WHEN log_date = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_WF,
    SUM(CASE WHEN log_date <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')        
        THEN ordered_units ELSE 0 END) as WF_MTD,
    SUM(CASE WHEN log_date <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')          
        THEN ordered_gmv ELSE 0 END) as GMV_WF_MTD 
    from y4a_sop_sell_out.sale_performance_by_channel_dpl
    where channel like 'WF%' and SKU in {list_sku} and country = 'USA'
    group by sku;
''', conn_post)  
df_WF.columns = df_WF.columns.str.upper()
#### Lấy data AMZ Seller
df_AMZ_seller = pd.read_sql(f'''
    select sku, 
    SUM(CASE WHEN log_date = '{data_time}' THEN ordered_units ELSE 0 END) as SELLASC,
    SUM(CASE WHEN log_date = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_ASC,
    SUM(CASE WHEN log_date <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')  
        THEN ordered_units ELSE 0 END) as ASC_MTD,
    SUM(CASE WHEN log_date <= '{data_time}'
        AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')          
        THEN ordered_gmv ELSE 0 END) as GMV_ASC_MTD   
    from y4a_sop_sell_out.sale_performance_by_channel_dpl
    where channel like 'ASC%'
    and sku in {list_sku}
    and country = 'USA'
    group by sku;
''', conn_post)
df_AMZ_seller.columns = df_AMZ_seller.columns.str.upper()

In [ ]:
#### Lấy ngày tiếp theo của real sales để lấy real time sales
max_data_time_1 = df_max_date + day
max_data_time_format = max_data_time_1.strftime("%d-%b-%y")
df_real_time_sales_MTD = pd.read_sql_query(f"""
    select asin, sum(ordered_units) as SellOutMTD, sum(ordered_revenue) as GMVOutMTD from 
    (select distinct asin, hour_pt , ordered_units, ordered_revenue  from dpl.ara_realtime_sales) T1
    where DATE_TRUNC('month',hour_pt) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day') and hour_pt >= '{max_data_time_format}%'
    and hour_pt < DATE_TRUNC('day', CURRENT_DATE)
group by asin """,conn_post_1) 
df_real_time_sales_MTD.columns = df_real_time_sales_MTD.columns.str.upper()
#### Lấy số sales trên real time sales ngày hôm qua
df_sell_out_today = pd.read_sql_query(f"""
    SELECT asin, SUM(ordered_units) AS SellOut, SUM(ordered_revenue) AS ordered_revenue
    FROM ( SELECT DISTINCT asin, hour_pt, ordered_units, ordered_revenue FROM dpl.ara_realtime_sales ) t1
    WHERE hour_pt BETWEEN CURRENT_DATE - interval '1 day' AND CURRENT_DATE - interval '1 hour'
    GROUP BY asin
    """,conn_post_1)
df_sell_out_today.columns = df_sell_out_today.columns.str.upper()

# Check data
data_sell_out_today = pd.read_sql_query(f"""
    SELECT ROUND(SUM(ordered_units)) AS SellOuttoday
    FROM dpl.ara_realtime_sales
    WHERE hour_pt BETWEEN CURRENT_DATE - interval '1 day' AND CURRENT_DATE - interval '1 hour' """,conn_post_1)
data_sell_out_avg30d = pd.read_sql_query(f"""
    SELECT ROUND(SUM(ordered_units)/30) AS Avg30daySellOut
    FROM dpl.ara_realtime_sales
    WHERE hour_pt BETWEEN CURRENT_DATE - interval '30 days' AND CURRENT_DATE - interval '1 hour' """,conn_post_1)

#### Lấy Info Asin
df_info_0 = pd.read_sql_query(f"""
    select SALES_PIC, ASIN, SKU,
    Case WHEN CATEGORY = 'Furniture - Indoor' AND PRODUCT_GROUP = 'Papasan Chair' THEN 'Papasan Chair'
         WHEN CATEGORY = 'Furniture - Indoor' AND PRODUCT_GROUP <> 'Papasan Chair' THEN 'Furniture - Indoor (excl PPS)' 
         ELSE CATEGORY END AS CATEGORY, PRODUCT_GROUP, VARIATIONS from
    (
    select distinct SALES_PIC,ASIN,SKU,
    Case WHEN CATEGORY = 'Services' THEN sub_category ELSE CATEGORY END AS CATEGORY,
    PRODUCT_GROUP, VARIATIONS from public.launching_plan
    where product_group is not null
    and asin in {list_asin}
    ) ABC
""",conn_post_1)
df_info = df_info_0.rename(columns={'sales_pic': 'Sales PIC','asin': 'ASIN', 'sku':'SKU' ,'category': 'Category','product_group': 'Product Group','variations': 'Product Name'})
def format_number(x):
    return "{:,.0f}".format(x)
def format_number_1(x):
    return "$" + "{:,.0f}".format(x)
def format_gap(x):
    return "{:.2%}".format(x)

In [ ]:
#### Lấy Data ACOS và Spend PPC MTD
asin_info = pd.read_sql_query(f"""
    select distinct CATEGORY,PRODUCT_GROUP from mi_report.launching_plan
    where PRODUCT_GROUP is not null
    and CATEGORY is not null
    """,conn_post_2)
asin_info.columns = asin_info.columns.str.upper()

df_ACOS_Spend = pd.read_sql_query(f"""
    with T1 as (
    select withdraw_time, date_, portfolio_name,  Spend, total_sales_14_day as Day_14_total_sales, campaign_id, campaign_name, ad_group_id, ad_group_name, 
    targeting, match_type, Impressions, clicks, total_orders_14_day as day_14_total_orders, total_units_14_day as day_14_total_units, amz_account, source_category as Type 
    from public.targeting
    where amz_account in ('A1AP6U5UUAP9DL','A2PXP9SSI7GDVG','A2U7X22FDY3QZO','AVXCWFLFGENR','A1ITUVWQWSB9L0')
    and date_ <= CURRENT_DATE - INTERVAL '1 day' AND date_ >= date_trunc('MONTH', CURRENT_DATE - INTERVAL '1 day')
    ),
    T2_MTD as (
    select portfolio_name, sum(Spend) as Spend_MTD, SUM(Day_14_total_sales) as Sales_MTD,
    CASE WHEN sum(Day_14_total_sales) = 0 THEN 0 ELSE sum(Spend)/sum(Day_14_total_sales) END as ACOS_MTD from T1
    where date_ <= CURRENT_DATE - INTERVAL '1 day' AND date_ >= date_trunc('MONTH', CURRENT_DATE - INTERVAL '1 day')
    group by portfolio_name
    ), T3_1 as (
    select portfolio_name, sum(Spend) as Spend_1, SUM(Day_14_total_sales) as Sales_1,
    CASE WHEN sum(Day_14_total_sales) = 0 THEN 0 ELSE sum(Spend)/sum(Day_14_total_sales) END as ACOS_1 from T1
    where date_ = CURRENT_DATE - INTERVAL '1 day'
    group by portfolio_name
    )
    select T2_MTD.portfolio_name, Spend_MTD, Sales_MTD, ACOS_MTD, Spend_1, Sales_1, ACOS_1 from T2_MTD
    left join T3_1 on T2_MTD.portfolio_name = T3_1.portfolio_name
    """,conn_post_2)
df_ACOS_Spend.columns = df_ACOS_Spend.columns.str.upper()
df_Por_Cat = pd.read_sql_query(f"""
    with t1 as (
    select distinct portfolio_name, advertised_asin from public.ad_products
        where amz_account in ('A1AP6U5UUAP9DL','A2PXP9SSI7GDVG','A2U7X22FDY3QZO','AVXCWFLFGENR','A1ITUVWQWSB9L0')
        and advertised_asin is not null
        and portfolio_name not in ('Test','Not grouped','-')
        and date_ >= CURRENT_DATE - INTERVAL '3 day'
    ), t2 as (
    Select portfolio_name, Category from t1
    left join
    (select asin, Category from mi_report.launching_plan) t3
    on t1.advertised_asin = t3.asin
    )
    select distinct portfolio_name, Category from t2
    where Category is not null
""",conn_post_2)
df_Por_Cat.columns = df_Por_Cat.columns.str.upper()

In [ ]:
#### Check Data ACOS và PPC MTD
Check_ACOS = df_ACOS_Spend[['SPEND_1']].dropna().sum()
if Check_ACOS[0] < 1000: 
    print('Chưa có data')
else:
    print('Đã có data')

In [ ]:
#### Tạo bảng ACOS - PPC - MTD
df_Spend = df_ACOS_Spend.merge(df_Por_Cat, on ='PORTFOLIO_NAME', how='left')

def categorize_row(row):
    if row['CATEGORY'] == 'Furniture - Indoor' and row['PORTFOLIO_NAME'] == '[Indoor Furniture] Papasan Chair':
        return 'Papasan Chair'
    elif row['CATEGORY'] == 'Furniture - Indoor' and row['PORTFOLIO_NAME'] != '[Indoor Furniture] Papasan Chair':
        return 'Furniture - Indoor (excl PPS)'
    else:
        return row['CATEGORY']

# Áp dụng hàm để tạo cột mới 'CATE OKR'
df_Spend['CATE OKR'] = df_Spend.apply(categorize_row, axis=1)

Spend_MTD = df_Spend.groupby('CATE OKR').agg({'SPEND_MTD':'sum','SALES_MTD':'sum'}).reset_index()

Spend_MTD.loc[len(Spend_MTD)] = ['Gardening + Outdoor Entertainment',
                                 Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Gardening', 'SPEND_MTD'].values[0]
                                 + Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Outdoor Entertainment', 'SPEND_MTD'].values[0],
                                 Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Gardening', 'SALES_MTD'].values[0]
                                 + Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Outdoor Entertainment', 'SALES_MTD'].values[0],]

Spend_MTD_1 = Spend_MTD[~Spend_MTD['CATE OKR'].isin(['Gardening', 'Outdoor Entertainment'])]

Spend_MTD_1['ACOS MTD'] = Spend_MTD_1['SPEND_MTD']/Spend_MTD_1['SALES_MTD']

Spend_MTD_2 = Spend_MTD_1.reset_index(drop=True)
Spend_MTD_2.loc[len(Spend_MTD_2)] = ['', Spend_MTD_2['SPEND_MTD'].sum(),Spend_MTD_2['SALES_MTD'].sum(),
                                         Spend_MTD_2['SPEND_MTD'].sum()/Spend_MTD_2['SALES_MTD'].sum(),]

Spend_MTD_3 = Spend_MTD_2.rename(columns={'SPEND_MTD': 'SPEND PPC MTD'})
#Spend_MTD_3.loc[Spend_MTD_3['CATEGORY'] == 'Services', 'CATEGORY'] = 'Services - Hoang Thong'

In [ ]:
#### Tạo bảng MTD sell out và Sell out
df_MTD = pd.concat([df_real_sales_MTD, df_real_time_sales_MTD])
df_MTD_sum = df_MTD.groupby(['ASIN']).agg({'SELLOUTMTD': 'sum','GMVOUTMTD': 'sum'}).reset_index() 
#### Table 1
df_sell_out_info = df_info.merge(df_MTD_sum, left_on ='ASIN', right_on= 'ASIN', how='left')
df_sell_out_info_0 = df_sell_out_info.merge(df_sell_out_today, left_on ='ASIN', right_on= 'ASIN', how='left')
df_sell_out_info_1 = df_sell_out_info_0.merge(df_WM, left_on ='SKU', right_on= 'SKU', how='left')
df_sell_out_info_2 = df_sell_out_info_1.merge(df_WF, left_on ='SKU', right_on= 'SKU', how='left')
df_sell_out_info_3 = df_sell_out_info_2.merge(df_AMZ_seller, left_on ='SKU', right_on= 'SKU', how='left')
df_sell_out_info_4 = df_sell_out_info_3.fillna(0)
df_sell_out_info_4['Total Orders Today'] = df_sell_out_info_4.apply(lambda row: row['SELLOUT'] + row['SELLWM'] + row['SELLWF'] + row['SELLASC'], axis=1)
df_sell_out_info_4['Total GMV Today'] = df_sell_out_info_4.apply(lambda row: row['ORDERED_REVENUE'] + row['GMV_WM'] + row['GMV_WF'] + row['GMV_ASC'], axis=1)
df_sell_out_info_4['Total Orders MTD'] = df_sell_out_info_4.apply(lambda row: row['SELLOUTMTD'] + row['WM_MTD'] + row['WF_MTD'] + row['ASC_MTD'], axis=1)
df_sell_out_info_4['Total GMV MTD'] = df_sell_out_info_4.apply(lambda row: row['GMVOUTMTD'] + row['GMV_WM_MTD'] + row['GMV_WF_MTD'] + row['GMV_ASC_MTD'], axis=1)

data_summary_1 = df_sell_out_info_4.groupby(['Sales PIC','Category', 'Product Group']).agg(
    {'SELLOUT': 'sum', 'ORDERED_REVENUE': 'sum', 'SELLOUTMTD': 'sum', 'GMVOUTMTD': 'sum', 
     'SELLWM': 'sum', 'GMV_WM': 'sum', 'WM_MTD': 'sum', 'GMV_WM_MTD': 'sum',
     'SELLWF': 'sum', 'GMV_WF':'sum', 'WF_MTD': 'sum', 'GMV_WF_MTD': 'sum', 
     'SELLASC': 'sum', 'ASC_MTD': 'sum', 'GMV_ASC':'sum', 'GMV_ASC_MTD': 'sum',
     'Total Orders Today': 'sum', 'Total GMV Today': 'sum', 'Total Orders MTD': 'sum', 'Total GMV MTD': 'sum', 
    }).reset_index()
data_summary_2 = data_summary_1.rename(columns=
    {'SELLOUT': 'AMZ AVC','ORDERED_REVENUE': 'GMV AMZ AVC', 'SELLOUTMTD': 'AMZ AVC MTD', 'GMVOUTMTD': 'GMV AMZ AVC MTD',
     'SELLWM': 'Walmart','GMV_WM': 'GMV Walmart', 'WM_MTD': 'Walmart MTD', 'GMV_WM_MTD': 'GMV Walmart MTD',
     'SELLWF': 'Wayfair','GMV_WF':'GMV Wayfair', 'WF_MTD': 'Wayfair MTD', 'GMV_WF_MTD': 'GMV Wayfair MTD',
     'SELLASC': 'AMZ ASC','GMV_ASC':'GMV AMZ ASC', 'ASC_MTD': 'AMZ ASC MTD', 'GMV_ASC_MTD': 'GMV AMZ ASC MTD',
    })
data_summary_3 = data_summary_2[['Sales PIC','Category','Product Group',
                                 'Total Orders MTD', 'AMZ AVC MTD', 'AMZ ASC MTD', 'Walmart MTD', 'Wayfair MTD',
                                 'Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                                 'Total Orders Today', 'AMZ AVC', 'AMZ ASC', 'Walmart', 'Wayfair',
                                 'Total GMV Today', 'GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']]
data_summary_4 = data_summary_3[(data_summary_3['Total Orders MTD'] != 0) | (data_summary_3['Total Orders Today'] != 0)]
data_summary_4 = data_summary_4.sort_values(['Category','Sales PIC'],ascending=[True,True])
    
# #### Table1.2
# data_summary_5 = data_summary_4[~((data_summary_4['Category'].str.contains('Services')) )]
#                                   #& (data_summary_4['Category'] != 'Services - Hoang Thong'))] 
# #### Table1.3
# data_summary_Services = data_summary_4[(data_summary_4['Category'].str.contains('Services')) ]
#                                           #& (data_summary_4['Category'] != 'Services - Hoang Thong')] 
def Row_Total(data): 
    data_1 = data.reset_index(drop=True)
    data_1.loc[len(data_1)] = ['','', 'Total', data_1['Total Orders MTD'].sum(), data_1['AMZ AVC MTD'].sum(),
                                           data_1['AMZ ASC MTD'].sum(), data_1['Walmart MTD'].sum(),
                                           data_1['Wayfair MTD'].sum(), data_1['Total GMV MTD'].sum(),
                                           data_1['GMV AMZ AVC MTD'].sum(), data_1['GMV AMZ ASC MTD'].sum(),
                                           data_1['GMV Walmart MTD'].sum(), data_1['GMV Wayfair MTD'].sum(),
                                           data_1['Total Orders Today'].sum(), data_1['AMZ AVC'].sum(),
                                           data_1['AMZ ASC'].sum(), data_1['Walmart'].sum(),
                                           data_1['Wayfair'].sum(), data_1['Total GMV Today'].sum(),
                                           data_1['GMV AMZ AVC'].sum(), data_1['GMV AMZ ASC'].sum(),
                                           data_1['GMV Walmart'].sum(), data_1['GMV Wayfair'].sum(),]

    data_1 = data_1.reindex([len(data_1) - 1] + list(range(len(data_1) - 1)))
    data_1[['Total Orders MTD','AMZ AVC MTD','AMZ ASC MTD','Walmart MTD','Wayfair MTD',
                'Total Orders Today','AMZ AVC','AMZ ASC','Walmart','Wayfair']] = data_1[
                ['Total Orders MTD','AMZ AVC MTD','AMZ ASC MTD','Walmart MTD','Wayfair MTD',
                'Total Orders Today','AMZ AVC','AMZ ASC','Walmart','Wayfair']].astype(int).applymap(format_number)
    data_1[['Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
               'Total GMV Today','GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']] = data_1[
                ['Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
               'Total GMV Today','GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']].astype(int).applymap(format_number_1)
    return data_1

data_summary_5 = Row_Total(data_summary_4)

# data_summary_Services_1 = Row_Total(data_summary_Services)

#### Table1.1
df_okr_cat = data_summary_4[['Category','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                            'GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']]

df_okr_cat_1 = df_okr_cat.groupby(['Category']).agg(
    {'GMV AMZ AVC MTD': 'sum', 'GMV AMZ ASC MTD': 'sum', 'GMV Walmart MTD': 'sum', 'GMV Wayfair MTD': 'sum', 
     'GMV AMZ AVC': 'sum', 'GMV AMZ ASC': 'sum', 'GMV Walmart': 'sum', 'GMV Wayfair': 'sum',
    }).reset_index()

df_okr_cat_AMZ_1 = df_okr_cat_1[['Category','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV AMZ AVC','GMV AMZ ASC']]
df_okr_cat_AMZ_1['GMV MTD'] = df_okr_cat_AMZ_1.apply(lambda row: row['GMV AMZ AVC MTD'] + row['GMV AMZ ASC MTD'] , axis=1)
df_okr_cat_AMZ_1['GMV Today'] = df_okr_cat_AMZ_1.apply(lambda row: row['GMV AMZ AVC'] + row['GMV AMZ ASC'] , axis=1)
df_okr_cat_AMZ_2 = df_okr_cat_AMZ_1[['Category','GMV MTD','GMV Today']]
df_okr_cat_AMZ_2.loc[len(df_okr_cat_AMZ_2)] = ['Gardening + Outdoor Entertainment',
                    df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Gardening', 'GMV MTD'].values[0]
                    + df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Outdoor Entertainment', 'GMV MTD'].values[0],
                    df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Gardening', 'GMV Today'].values[0]
                    + df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Outdoor Entertainment', 'GMV Today'].values[0]]
df_okr_cat_AMZ_3 = df_okr_cat_AMZ_2[~df_okr_cat_AMZ_2['Category'].isin(['Gardening', 'Outdoor Entertainment'])]

df_okr_cat_WM_1 = df_okr_cat_1[['Category','GMV Walmart MTD','GMV Walmart']]
df_okr_cat_WM_1.loc[len(df_okr_cat_WM_1)] = ['Walmart',
                                             df_okr_cat_WM_1['GMV Walmart MTD'].sum(),df_okr_cat_WM_1['GMV Walmart'].sum()]
df_okr_cat_WM_2 = df_okr_cat_WM_1[df_okr_cat_WM_1['Category']=='Walmart']
df_okr_cat_WM_3 = df_okr_cat_WM_2.rename(columns={'GMV Walmart MTD': 'GMV MTD','GMV Walmart': 'GMV Today', })

df_okr_cat_WF_1 = df_okr_cat_1[['Category','GMV Wayfair MTD','GMV Wayfair']]
df_okr_cat_WF_1.loc[len(df_okr_cat_WF_1)] = ['Wayfair',
                                             df_okr_cat_WF_1['GMV Wayfair MTD'].sum(), df_okr_cat_WF_1['GMV Wayfair'].sum(),]
df_okr_cat_WF_2 = df_okr_cat_WF_1[df_okr_cat_WF_1['Category']=='Wayfair']
df_okr_cat_WF_3 = df_okr_cat_WF_2.rename(columns={'GMV Wayfair MTD': 'GMV MTD','GMV Wayfair': 'GMV Today', })

df_okr_cat_2 = pd.concat([df_okr_cat_AMZ_3, df_okr_cat_WM_3, df_okr_cat_WF_3], ignore_index=True)
df_okr_cat_2['Category'] = df_okr_cat_2['Category'].apply(lambda x: 'Services' if 'Services' in x else x)
df_okr_cat_3 = df_okr_cat_2.groupby(df_okr_cat_2['Category']).sum().reset_index()

df_okr_cat_4 = df_okr_cat_3.reset_index(drop=True)
df_okr_cat_4.loc[len(df_okr_cat_4)] = ['', df_okr_cat_4['GMV MTD'].sum(), df_okr_cat_4['GMV Today'].sum(),]
df_okr_cat_5 = df_OKR_GG.merge(df_okr_cat_4, left_on ='Category', right_on= 'Category', how='left')
#     df_okr_cat_5 = df_okr_cat_5.reindex([len(df_okr_cat_5) - 1] + list(range(len(df_okr_cat_5) - 1)))
df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].replace('[^\d.]', '', regex=True)
df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].replace(',', '', regex=True)
df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].astype(float)

df_okr_cat_5['% GMV MTD / OKR MTD'] = df_okr_cat_5.apply(lambda row: row['GMV MTD'] / row['OKR MTD'] 
                                                           if row['Total Order OKR'] != 0 else "-", axis=1)

df_okr_cat_5['% GMV Today / OKR Today'] = df_okr_cat_5.apply(lambda row: row['GMV Today'] / row['OKR Today']
                                                             if row['OKR Today'] != 0 else "-", axis=1)

# Ghép với bảng ACOS - PPC - MTD
df_okr_cat_6 = df_okr_cat_5.merge(Spend_MTD_3, left_on ='Category', right_on = 'CATE OKR', how='left')

df_okr_cat_7 = df_okr_cat_6.fillna(0)

df_okr_cat_8 = df_okr_cat_7[['Market Place','Category','Total Order OKR','GMV MTD','OKR MTD','% GMV MTD / OKR MTD',
                             'SPEND PPC MTD','ACOS MTD','GMV Today','OKR Today','% GMV Today / OKR Today']]

df_okr_cat_8[['Total Order OKR','GMV MTD','OKR MTD','GMV Today','OKR Today','SPEND PPC MTD']] = df_okr_cat_8[
['Total Order OKR','GMV MTD','OKR MTD','GMV Today','OKR Today','SPEND PPC MTD']].astype(int).applymap(format_number)

df_okr_cat_8[['% GMV MTD / OKR MTD','% GMV Today / OKR Today','ACOS MTD']] = df_okr_cat_8[
['% GMV MTD / OKR MTD','% GMV Today / OKR Today','ACOS MTD']].applymap(lambda x: format_gap(x) if x != "-" else x)

df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'Category'] = ' '
df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'Category'] = ' '
df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'SPEND PPC MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'SPEND PPC MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'ACOS MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'ACOS MTD'] = 'No data'
#Hard Code Off-site
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'GMV MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', '% GMV MTD / OKR MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'SPEND PPC MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'ACOS MTD'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'GMV Today'] = 'No data'
df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', '% GMV Today / OKR Today'] = 'No data'

df_okr_cat_9 = df_okr_cat_8[df_okr_cat_8['Market Place']!='']

#### Get Target MTD and Daily 
Table_ORK = df_OKR_GG[df_OKR_GG['Category'] == 'Total']
Table_ORK_1 = Table_ORK.reset_index(drop=True)
OKR_OCT = Table_ORK_1['Total Order OKR'].replace('[^\d.]', '', regex=True).astype(float)
OKR_Target = Table_ORK_1['OKR Team'].replace('[^\d.]', '', regex=True).astype(float)
OKR_Target_Base = Table_ORK_1['OKR Base'].replace('[^\d.]', '', regex=True).astype(float)
OKR_Target_MTD = Table_ORK_1['OKR MTD'].replace('[^\d.]', '', regex=True).astype(float)

#### Alert OKR
# Lấy Time Gone
day_of_month = data_date.day
month = data_date.month
year = data_date.year

# Sử dụng module calendar để lấy số ngày trong tháng
_, num_days = calendar.monthrange(year, month)

Time_gone = day_of_month/num_days    
GMV_today = data_summary_4['Total GMV Today'].sum()
GMV_MTD = data_summary_4['Total GMV MTD'].sum()
Target_OKR = format_number_1(OKR_OCT[0])

if GMV_MTD > OKR_Target_MTD[0]:
    OKR_Alert = OKR_Target_Base[0]
    Gap = GMV_today / OKR_Target_Base[0]
else:
    OKR_Alert = OKR_Target[0]
    Gap = GMV_today / OKR_Target[0]

Gap_MTD = GMV_MTD / OKR_OCT[0]
Gap_per = format_gap(Gap)
Gap_MTD_per = format_gap(Gap_MTD)
Time_gone_per = format_gap(Time_gone)

In [ ]:
data_summary_5

In [ ]:
df_okr_cat_9

In [ ]:
# data_summary_Services_1

In [ ]:
# Table 2
data_detail_sell_in = df_detail_sell_in.merge(df_info, left_on ='ASIN', right_on= 'ASIN', how='left')

data_detail_sell_in_1 = data_detail_sell_in.groupby(['Sales PIC','Category', 'Product Group']).agg(
    {'AVCDS': 'sum', 'AVCWH': 'sum', 'AVCDI': 'sum','ALLCHANNEL': 'sum'}).reset_index()

data_detail_sell_in_2 = data_detail_sell_in_1.rename(columns=
    {'AVCDS': 'AVC DS', 'AVCWH': 'AVC WH','AVCDI': 'AVC DI',
     'ALLCHANNEL': 'All Channel'})

data_detail_sell_in_2 = data_detail_sell_in_2.sort_values(['Category','Sales PIC'],ascending=[True,True])

data_detail_sell_in_2.loc[len(data_detail_sell_in_2)] = ['','', 'Tổng', data_detail_sell_in_2['AVC DS'].sum(), 
                            data_detail_sell_in_2['AVC WH'].sum(),
                            data_detail_sell_in_2['AVC DI'].sum(),
                            data_detail_sell_in_2['All Channel'].sum()]

data_detail_sell_in_2[['AVC DS','AVC WH','AVC DI','All Channel']] = data_detail_sell_in_2[
    ['AVC DS','AVC WH','AVC DI','All Channel']].astype(int).applymap(format_number)

data_detail_sell_in_3 = data_detail_sell_in_2[data_detail_sell_in_2['Category']!="#N/A"]
data_detail_sell_in_4 = data_detail_sell_in_3[data_detail_sell_in_3['All Channel']!="0"]

In [ ]:
data_detail_sell_in_4

In [ ]:
##### Điều kiện 1: SellOut today > 50% SellOut Avg 30day
sell_out_today = data_sell_out_today.iloc[0]['sellouttoday']
sell_out_avg30d = data_sell_out_avg30d.iloc[0]['avg30daysellout']
#    return print(data_summary_2,data_detail_sell_in_2)

if sell_out_today <= sell_out_avg30d*0:
    # Tổng hợp thông tin nếu có lỗi
    print('Data Real Time Sales Có lỗi')
    print(f'Tổng sales out today: {sell_out_today}')
    print(f'80% Tổng số sales out trong 30 ngày gần nhất: {sell_out_avg30d*0.5}')
    data_summary_2
    data_detail_sell_in_2
else:
    print('Data Bình thường')

In [ ]:
####### Đổi màu tên cột
def format_name_color(table, col):
    first_row_t1 = table.find_all('tr')[1]
    cell = first_row_t1.find_all('th')[col]
    cell['style'] = cell['style'].replace('#548235', '#305496')
    return table
###### Đổi màu tên cột services
def format_name_color_services(table, col):
    first_row_t1 = table.find_all('tr')[0]
    cell = first_row_t1.find_all('th')[col]
    cell['style'] = cell['style'].replace('#548235', '#305496')
    return table       
######## Đổi màu cột
def format_color(table, col):
    table_1 = table.select(f'td:nth-child({col})')
    for cell in table_1:
        cell['style'] = cell['style'].replace('#E2EFDA', '#D9E1F2')
    return table
######## Căn các dòng bằng nhau (70px)
def format_width(table, col):
    table_1 = table.select(f'td:nth-child({col})')
    for cell in table_1:
        cell['style'] = cell['style'] + ';width: 70px'
    return table        
def conditional_formatting(table, col_data_4, col_data_5, Time_gone):
    # Select the data cells in the specified columns for the first row
    data_cells_4_row1 = table.select(f'tr:nth-child(1) td:nth-child({col_data_4})')
    # Apply background color to the first row based on the condition for column 4
    for cell in data_cells_4_row1:
        text = cell.text.strip()
        value = re.findall(r'\d+\.\d+|\d+', text)
        if value and float(value[0]) < Time_gone*100:
            cell['style'] += ';' + 'background-color: #e06666'
        elif value and float(value[0]) >= Time_gone*100:
            cell['style'] += ';' + 'background-color: #b7e1cd'
        else:
            pass
    # Select the data cells in the specified column for rows 2 and onward
    data_cells_5_remaining_rows = table.select(f'tr:nth-child(n+2) td:nth-child({col_data_5})')

    for cell in data_cells_5_remaining_rows:
        text = cell.text.strip()
        value = re.findall(r'\d+\.\d+|\d+', text)
        if value and float(value[0]) < Time_gone*100:
            cell['style'] += ';' + 'background-color: #e06666'
        elif value and float(value[0]) >= Time_gone*100:
            cell['style'] += ';' + 'background-color: #b7e1cd'
        else:
            pass          
    return table
def conditional_formatting_ACOS(table, col):
        table_1 = table.select(f'td:nth-child({col})')
        for cell in table_1:
            text = cell.text.strip()  # Xóa khoảng trắng ở đầu và cuối chuỗi
            value = re.findall(r'\d+\.\d+|\d+', text)  # Trích xuất số từ chuỗi
            if value and float(value[0]) <= 20:
                cell['style'] += ';' + 'background-color: #b7e1cd'
            elif value and float(value[0]) <= 40:
                cell['style'] += ';' + 'background-color: #fff2cc'
            elif value and float(value[0]) <= 70:
                cell['style'] += ';' + 'background-color: #f4cccc' 
            elif value and float(value[0]) > 70 :
                cell['style'] += ';' + 'background-color: #e06666'
            elif text == 'Zero Sales' or text == 'Zero Sales Ads':
                cell['style'] += ';' + 'background-color: #e06666'
            elif text == 'NLY' or text == 'No Spent' or text == 'NA' or text == 'No data':
                cell['style'] += ';' + 'background-color: #b7b7b7'
            else:
                pass
        table_2 = str(table)

        return table
######## Table Category
table_category = build_table(df_okr_cat_9, 'green_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='3px 3px 3px 3px', width='auto',) 

table_category_1 = BeautifulSoup(table_category, 'html.parser')  

#### Tô màu tên cột
for i in (2,3,4,5,6,7):
    table_category_2 = format_name_color_services(table_category_1, i)  

#### Tô màu cột
for i in (3,4,5,6,7,8):
    table_category_2 = format_color(table_category_1, i)  

#table_category_3 = conditional_formatting(table_category_2, 5,Time_gone)
table_category_3 = conditional_formatting(table_category_2, 6, 6, 1)
table_category_3 = conditional_formatting(table_category_2, 11, 11, 1)
table_category_3 = conditional_formatting_ACOS(table_category_2, 8)
table_category_4 = str(table_category_3)

######## Table Product   
header_col_table_product = list(data_summary_5.columns.values)
col_table_product = ['','','','','','','','Total OKR',Target_OKR,Gap_MTD_per,'Time Gone',Time_gone_per,'',
#'Total OKR',Target_OKR,'N/A','Time Gone','N/A','',
'','','','',f'Target {formatted_time_daily}',format_number_1(OKR_Alert),Gap_per,'','','',]
mapping_table_product = zip(col_table_product, header_col_table_product)
col_list_table_product = pd.MultiIndex.from_tuples(mapping_table_product)
data_summary_5.columns = col_list_table_product

table_product = build_table(data_summary_5, 'green_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='3px 3px 3px 3px', width='auto',)

#### Gộp cell Tổng            
replacements_pro = {
'<td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto">Total</td>':
'<td colspan="3" halign="center" style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Total</td>',
}

for old_value, new_value in replacements_pro.items():
    table_product = table_product.replace(old_value, new_value)

table_product_1 = BeautifulSoup(table_product, 'html.parser')  

#### Tô màu hàng Tổng
last_row_pro = table_product_1.find_all('tr')[2]
cells_last_row_pro = last_row_pro.find_all('td')
for cell in cells_last_row_pro:
    cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''

#### Tô màu Alert
first_row_t1 = table_product_1.find_all('tr')[0]
cells_first_row_t1_OKR = first_row_t1.find_all('th')[8]
cells_first_row_t1_per_OKR = first_row_t1.find_all('th')[9]
cells_first_row_t1_gap_OKR = first_row_t1.find_all('th')[5]
cells_first_row_t1_gap_Time_gone = first_row_t1.find_all('th')[3]

cells_first_row_t1_OKR['style'] = cells_first_row_t1_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
cells_first_row_t1_per_OKR['style'] = cells_first_row_t1_per_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
cells_first_row_t1_gap_OKR['style'] = cells_first_row_t1_gap_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
cells_first_row_t1_gap_Time_gone['style'] = cells_first_row_t1_gap_Time_gone['style'].replace('border-bottom: 2px solid #548235;', '') 

if GMV_today >= OKR_Alert:
    cells_first_row_t1_OKR['style'] += ';' + 'background-color: #0a6b3d'
    cells_first_row_t1_per_OKR['style']+= ';' + 'background-color: #0a6b3d'
else:
    cells_first_row_t1_OKR['style'] += ';' + 'background-color: #e06666'
    cells_first_row_t1_per_OKR['style'] += ';' + 'background-color: #e06666'

if Gap_MTD >= Time_gone:
    cells_first_row_t1_gap_OKR['style'] += ';' + 'background-color: #0a6b3d'
    cells_first_row_t1_gap_Time_gone['style'] += ';' + 'background-color: #0a6b3d'
else:
    cells_first_row_t1_gap_OKR['style'] += ';' + 'background-color: #e06666'
    cells_first_row_t1_gap_Time_gone['style'] += ';' + 'background-color: #e06666'

#### Tô màu cột
for i in (3,4,5,6,7,13,14,15,16,17):
    table_product_2 = format_name_color(table_product_1, i)  

for i in (4,5,6,7,8,14,15,16,17,18):
    table_product_2 = format_color(table_product_1, i)    

for i in range(4,24):
    table_product_2 = format_width(table_product_1, i)  

table_product_3 = str(table_product_2)

# ######## Table_servie
# table_services = build_table(data_summary_Services_1, 'green_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='3px 3px 3px 3px', width='auto',)       

# #### Gộp cell Tổng            
# replacements_ser = {
# '<td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto">Total</td>':
# '<td colspan="3" halign="center" style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Total</td>',
# }

# for old_value, new_value in replacements_ser.items():
#     table_services = table_services.replace(old_value, new_value)

# table_services_1 = BeautifulSoup(table_services, 'html.parser')  

# #### Tô màu hàng Tổng
# last_row_ser = table_services_1.find_all('tr')[1]
# cells_last_row_ser = last_row_ser.find_all('td')
# for cell in cells_last_row_ser:
#     cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''

# for i in (3,4,5,6,7,13,14,15,16,17):
#     table_services_2 = format_name_color_services(table_services_1, i)  

# for i in (4,5,6,7,8,14,15,16,17,18):
#     table_services_2 = format_color(table_services_1, i)    

# for i in range(4,24):
#     table_services_2 = format_width(table_services_1, i)  

# table_services_3 = str(table_services_2)


######## Table2 = data_detail_sell_in_2.to_html(index=False)
table2 = build_table(data_detail_sell_in_4,'orange_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='4px 4px 4px 4px')

if data_detail_sell_in_4.empty: 
    table2_2 = '<b> Không Có Data Sell In </b>'
else:
    table2_1 = BeautifulSoup(table2, 'html.parser')  
#### Tô màu hàng cuối
    last_row_t2 = table2_1.find_all('tr')[-1]
    cells_last_row_t2 = last_row_t2.find_all('td')
    for cell in cells_last_row_t2:
        cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''

    table2_2 = str(table2_1)

In [ ]:
######### Thiết lập thông tin tài khoản email
email = 'hieptn@yes4all.com'
password = 'bqaw gtob iobs zpal'
######### Thiết lập đối tượng email
msg = MIMEMultipart()

#         recipients = ['deploymentlab@yes4all.com','thuyle@yes4all.com', 'hungvo@yes4all.com','dinhnd@yes4all.com',
#                       'tranghh@yes4all.com','tuanta@yes4all.com','tienttt@yes4all.com','ngocntm@yes4all.com',
#                       'vylm@yes4all.com','leader_innonet@yes4all.com']
#         cc_recipients = ['thuyle@yes4all.com', 'hungvo@yes4all.com','dinhnd@yes4all.com','tranghh@yes4all.com',
#                          'tuanta@yes4all.com','tienttt@yes4all.com','ngocntm@yes4all.com','vylm@yes4all.com',
#                         'leader_innonet@yes4all.com']
#         msg['To'] = 'deploymentlab@yes4all.com'
#         msg['CC'] = ", ".join(cc_recipients)

#         recipients = ['hieppropm224@gmail.com','hungvo@yes4all.com','vylm@yes4all.com']
#         msg['To'] = ", ".join(recipients)

recipients = 'hieptn@yes4all.com'
msg['To'] = 'hieptn@yes4all.com'

msg['From'] = email 
msg['Subject'] = f'DAILY SALES PERFORMANCE UPDATE - {Subject_time}'

######### Tạo nội dung HTML
html = f"""
<html>
    <body>
        <p>Dear team,</p>
        <p>Em xin update mail <b>Daily Performance</b> của team mình <b>{formatted_time}</b> cho 
        các project NPD và SLOB như sau:</p>
        <h3><font color="Blue"><b>A. SELL OUT </b></font></h3> 
        <h4><font color="Blue"><b>I. CATEGORY </b></font></h4> 
            {table_category_4}
        <h4><font color="Blue"><b>II. PRODUCT GROUP </b></font></h4> 
        <p>AMZ AVC: Amazon Vender Central<br>
        AMZ ASC: Amazon Seller Central</p>
            {table_product_3} 
        <h3><font color="Blue"><b>B. SALE IN </b></font></h3>
            {table2_2}     
        <p>Cảm ơn Team.</p>

        <p>Best Regards,<br>
        Trần Ngọc Hiệp<br>
        Data Analyst<br>
        Phone number: 032 9704 588<br>
        Email:        hieptn@yes4all.com </p>             
    </body>
</html>
"""
######### Chuyển đổi nội dung HTML thành MIMEText
text = MIMEText(html, 'html')
msg.attach(text)
######### Thiết lập kết nối SMTP và gửi email
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login(email, password)
server.sendmail(email, recipients, msg.as_string())
server.quit()
########sys.exit()
print("Job executed at", datetime.datetime.now())
print('Đã gửi mail daily')

## Chạy Hàm

### Hàm

In [4]:
#Tạo hàm
def Auto_mail_daily():    
#### Lấy data postgresql
    conn_post = psycopg2.connect(host="172.30.105.112",database="y4a_datamart",user="y4a_dl_vylm",password="ACB!asdlkj123")
#### Data BI (Quang Ù)
    conn_post_1 = psycopg2.connect(host="172.29.15.98",database="mi_report", user="da_report",password="Inno202$")
#### Data BI (Quang Ù) -- Spend Daily
    conn_post_2 = psycopg2.connect(host="172.29.15.98",database="bi_sem", user="da_report",password="Inno202$")
#### Connet Google Sheet get OKR
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(r"G:\Shared drives\Deployment Data Template\Power BI\Python Code\2.Daily\email-sending-via-python-fd13e6126e20.json", scope)
    client = gspread.authorize(creds)
    service = build('sheets', 'v4', credentials=creds)
    sheet_target_OKR = '173CwhjxOkark9N4UTJJiiD6a7eN6IvUWcA4zDLs5l1w'
    #https://docs.google.com/spreadsheets/d/173CwhjxOkark9N4UTJJiiD6a7eN6IvUWcA4zDLs5l1w/edit#gid=2125877975
    range_name_OKR = 'Target OKR Feb 24!B3:K'
    result_OKR = service.spreadsheets().values().get(
    spreadsheetId=sheet_target_OKR, range=range_name_OKR).execute()
    values_OKR = result_OKR.get('values', [])
    df_OKR_GG = pd.DataFrame(result_OKR['values'][1:], columns=result_OKR['values'][0])
#### Lấy list Asin, SKU
    list_asin_sku = pd.read_sql_query(f"""
        select asin, sku from public.launching_plan where asin is not null and sku is not null """,conn_post_1)
#### List_asin
    list_asin = tuple(list_asin_sku['asin'])
#### List_sku
    list_sku = tuple(list_asin_sku['sku'])
#### Lấy ngày data
    now = datetime.datetime.now()
    day = datetime.timedelta(days=1)
    data_date = now - day
    data_time = data_date.strftime("%Y-%m-%d")
    print(f'data time: {data_time}')
    formatted_time = data_date.strftime("ngày %d tháng %m")
    formatted_time_daily = data_date.strftime("%d-%b")
    Subject_time = data_date.strftime("%d %b").upper()
####Lấy ngày có data real sales mới nhất
    df_max_date_real_sales = pd.read_sql(f'''
         select MAX(log_date) as MAX_DAY from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where channel in ('AVC WH','AVC DI') and label_datetime = 'daily' and asin in {list_asin}  
        ''', con=conn_post)
    #and log_date <= '2023-12-07'
    df_max_date = df_max_date_real_sales.iloc[0]['max_day']
    max_data_time = df_max_date.strftime("%Y-%m-%d")
    max_data_time_mysql = df_max_date.strftime("%d-%m-%Y")
####Lấy data ngày có real sales mới nhất
    df_real_sales_MTD = pd.read_sql(f'''
        select asin as ASIN, 
            SUM(CASE WHEN log_date <= '{max_data_time}' 
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
                THEN ordered_units ELSE 0 END) as SELLOUTMTD,
            SUM(CASE WHEN log_date <= '{max_data_time}' 
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
                THEN ordered_gmv ELSE 0 END) as GMVOUTMTD    
        from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where ASIN in {list_asin}
        and country = 'USA' and channel not like 'WM%' and channel not like 'WF%' and channel not like 'ASC%'
        group by ASIN;
    ''', con=conn_post)
    df_real_sales_MTD.columns = df_real_sales_MTD.columns.str.upper()
#### Lấy data sell in
    df_sell_in = pd.read_sql(f'''
        SELECT Asin,  
            SUM(CASE WHEN date <= '{data_time}'     
                AND DATE_TRUNC('month', date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
                AND DATE_TRUNC('year', date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
                THEN ordered_unit ELSE 0 END) AS SellInMTD, 
            SUM(CASE WHEN date = '{data_time}'and channel in ('AVC DS','AVC WH','AVC DI') THEN ordered_unit
                ELSE 0 END) AS SellIn
        FROM y4a_sop_sell_in.sell_in_perfomance_ordered_date_by_location
        where asin in {list_asin}
        GROUP BY Asin; ''', con=conn_post)
    df_sell_in.columns = df_sell_in.columns.str.upper()
    #### Lấy data sell in detail
    df_detail_sell_in = pd.read_sql(f'''
        SELECT Asin,  
            SUM(CASE WHEN date = '{data_time}' and channel = 'AVC DS' THEN ordered_unit ELSE 0 END) AS AVCDS,
            SUM(CASE WHEN date = '{data_time}' and channel = 'AVC WH' THEN ordered_unit ELSE 0  END) AS AVCWH,
            SUM(CASE WHEN date = '{data_time}' and channel = 'AVC DI' THEN ordered_unit ELSE 0  END) AS AVCDI,
            SUM(CASE WHEN date = '{data_time}' and channel in ('AVC DS','AVC WH','AVC DI') THEN ordered_unit ELSE 0 END) AS AllChannel
        FROM y4a_sop_sell_in.sell_in_perfomance_ordered_date_by_location
        where asin in {list_asin}
        GROUP BY Asin; ''', con=conn_post)
    df_detail_sell_in.columns = df_detail_sell_in.columns.str.upper()
#### Lấy data WM bảng Postgre
    df_WM = pd.read_sql_query(f'''
        with T1 as (
        select DATE_TRUNC('day', Cover_order_date) as log_date, left(item_sku,4) as sku, Count(order_line_id) as ordered_units, SUM(product_charge_amt) as ordered_gmv from (
        SELECT order_date, order_date - INTERVAL '7 hours' as Cover_order_date, order_line_id, item_sku, product_charge_amt, item_product_name FROM y4a_inno.tb_dpl_wm_dsv_order_rev_dtl
        where left(item_sku,4) in {list_sku}
        order by order_date desc ) ABC
        where DATE_TRUNC('month', Cover_order_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', Cover_order_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
        group by DATE_TRUNC('day', Cover_order_date), sku
        order by log_date desc
        ), T2 as (
        select DATE_TRUNC('day', Cover_order_date) as log_date, left(sku,4) as sku, sum(qty) as ordered_units , sum(prd_charge) from (
        select order_date - INTERVAL '7 hours' as Cover_order_date, sku, qty, prd_charge from y4a_inno.tb_dpl_wm_wsc_ord_dtl_by_sku
        where left(sku,4) in {list_sku}
        and sku_status <> 'Cancelled'
        order by order_date desc ) ABCD
        Where DATE_TRUNC('month', Cover_order_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
        AND DATE_TRUNC('year', Cover_order_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
        group by log_date, sku
        order by log_date
        ), T3 as ( 
        SELECT * FROM T1
        UNION
        SELECT * FROM T2
        ) 
        select sku, 
        SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') = '{data_time}' THEN ordered_units ELSE 0 END) as SELLWM,
        SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_WM,
        SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
            THEN ordered_units ELSE 0 END) as WM_MTD,
        SUM(CASE WHEN TO_CHAR(log_date, 'YYYY-MM-DD') <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')
            THEN ordered_gmv ELSE 0 END) as GMV_WM_MTD
        from T3
        group by sku ''', conn_post)
    df_WM.columns = df_WM.columns.str.upper()
#### Lấy data Wayfair
    df_WF = pd.read_sql(f'''
        select sku, 
        SUM(CASE WHEN log_date = '{data_time}' THEN ordered_units ELSE 0 END) as SELLWF,
        SUM(CASE WHEN log_date = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_WF,
        SUM(CASE WHEN log_date <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')        
            THEN ordered_units ELSE 0 END) as WF_MTD,
        SUM(CASE WHEN log_date <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')          
            THEN ordered_gmv ELSE 0 END) as GMV_WF_MTD 
        from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where channel like 'WF%' and SKU in {list_sku} and country = 'USA'
        group by sku; ''', conn_post)  
    df_WF.columns = df_WF.columns.str.upper()
    #### Lấy data AMZ Seller
    df_AMZ_seller = pd.read_sql(f'''
        select sku, 
        SUM(CASE WHEN log_date = '{data_time}' THEN ordered_units ELSE 0 END) as SELLASC,
        SUM(CASE WHEN log_date = '{data_time}' THEN ordered_gmv ELSE 0 END) as GMV_ASC,
        SUM(CASE WHEN log_date <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')  
            THEN ordered_units ELSE 0 END) as ASC_MTD,
        SUM(CASE WHEN log_date <= '{data_time}'
            AND DATE_TRUNC('month', log_date) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day')
            AND DATE_TRUNC('year', log_date) = DATE_TRUNC('year', CURRENT_DATE - INTERVAL '1 day')          
            THEN ordered_gmv ELSE 0 END) as GMV_ASC_MTD   
        from y4a_sop_sell_out.sale_performance_by_channel_dpl
        where channel like 'ASC%'
        and sku in {list_sku}
        and country = 'USA'
        group by sku; ''', conn_post) 
    df_AMZ_seller.columns = df_AMZ_seller.columns.str.upper()
#### Lấy ngày tiếp theo của real sales để lấy real time sales
    max_data_time_1 = df_max_date + day
    max_data_time_format = max_data_time_1.strftime("%d-%b-%y")
    df_real_time_sales_MTD = pd.read_sql_query(f"""
    select asin, sum(ordered_units) as SellOutMTD, sum(ordered_revenue) as GMVOutMTD from 
    (select distinct asin, hour_pt , ordered_units, ordered_revenue  from dpl.ara_realtime_sales) T1
    where DATE_TRUNC('month',hour_pt) = DATE_TRUNC('month', CURRENT_DATE - INTERVAL '1 day') and hour_pt >= '{max_data_time_format}%'
    and hour_pt < DATE_TRUNC('day', CURRENT_DATE)
    group by asin """,conn_post_1) 
    df_real_time_sales_MTD.columns = df_real_time_sales_MTD.columns.str.upper()
#### Lấy số sales trên real time sales ngày hôm qua
    df_sell_out_today = pd.read_sql_query(f"""
        SELECT asin, SUM(ordered_units) AS SellOut, SUM(ordered_revenue) AS ordered_revenue
        FROM ( SELECT DISTINCT asin, hour_pt, ordered_units, ordered_revenue FROM dpl.ara_realtime_sales ) t1
        WHERE hour_pt BETWEEN CURRENT_DATE - interval '1 day' AND CURRENT_DATE - interval '1 hour'
        GROUP BY asin
        """,conn_post_1)
    df_sell_out_today.columns = df_sell_out_today.columns.str.upper()    
#### Check data
    data_sell_out_today = pd.read_sql_query(f"""
        SELECT ROUND(SUM(ordered_units)) AS SellOuttoday
        FROM dpl.ara_realtime_sales
        WHERE hour_pt BETWEEN CURRENT_DATE - interval '1 day' AND CURRENT_DATE - interval '1 hour' """,conn_post_1)
    data_sell_out_avg30d = pd.read_sql_query(f"""
        SELECT ROUND(SUM(ordered_units)/30) AS Avg30daySellOut
        FROM dpl.ara_realtime_sales
        WHERE hour_pt BETWEEN CURRENT_DATE - interval '30 days' AND CURRENT_DATE - interval '1 hour' """,conn_post_1)
#### Lấy Info Asin
    df_info_0 = pd.read_sql_query(f"""
    select SALES_PIC, ASIN, SKU,
        Case WHEN CATEGORY = 'Furniture - Indoor' AND PRODUCT_GROUP like '%Papasan Chair' THEN 'Papasan Chair'
             WHEN CATEGORY = 'Furniture - Indoor' AND PRODUCT_GROUP not like '%Papasan Chair' THEN 'Furniture - Indoor (excl PPS)' 
             ELSE CATEGORY END AS CATEGORY, PRODUCT_GROUP, VARIATIONS from
        ( select distinct SALES_PIC,ASIN,SKU,
        Case WHEN CATEGORY = 'Services' THEN sub_category ELSE CATEGORY END AS CATEGORY,
        PRODUCT_GROUP, VARIATIONS from public.launching_plan
        where product_group is not null
        and asin in {list_asin} ) ABC
    """,conn_post_1)
    df_info = df_info_0.rename(columns={'sales_pic': 'Sales PIC','asin': 'ASIN', 'sku':'SKU' ,'category': 'Category','product_group': 'Product Group','variations': 'Product Name'})
    def format_number(x):
        return "{:,.0f}".format(x)
    def format_number_1(x):
        return "$" + "{:,.0f}".format(x)
    def format_gap(x):
        return "{:.2%}".format(x)
#### Lấy Data ACOS và Spend PPC MTD
    # asin_info = pd.read_sql_query(f"""
    #     select distinct CATEGORY,PRODUCT_GROUP from mi_report.launching_plan
    #     where PRODUCT_GROUP is not null and CATEGORY is not null
    #     """,conn_post_2)
    # asin_info.columns = asin_info.columns.str.upper()
    # df_ACOS_Spend = pd.read_sql_query(f"""
    #     with T1 as (
    #     select withdraw_time, date_, portfolio_name,  Spend, total_sales_14_day as Day_14_total_sales, campaign_id, campaign_name, ad_group_id, ad_group_name, 
    #     targeting, match_type, Impressions, clicks, total_orders_14_day as day_14_total_orders, total_units_14_day as day_14_total_units, amz_account, source_category as Type 
    #     from public.targeting
    #     where amz_account in ('A1AP6U5UUAP9DL','A2PXP9SSI7GDVG','A2U7X22FDY3QZO','AVXCWFLFGENR','A1ITUVWQWSB9L0')
    #     and date_ <= CURRENT_DATE - INTERVAL '1 day' AND date_ >= date_trunc('MONTH', CURRENT_DATE - INTERVAL '1 day')
    #     ),
    #     T2_MTD as (
    #     select portfolio_name, sum(Spend) as Spend_MTD, SUM(Day_14_total_sales) as Sales_MTD,
    #     CASE WHEN sum(Day_14_total_sales) = 0 THEN 0 ELSE sum(Spend)/sum(Day_14_total_sales) END as ACOS_MTD from T1
    #     where date_ <= CURRENT_DATE - INTERVAL '1 day' AND date_ >= date_trunc('MONTH', CURRENT_DATE - INTERVAL '1 day')
    #     group by portfolio_name
    #     ), T3_1 as (
    #     select portfolio_name, sum(Spend) as Spend_1, SUM(Day_14_total_sales) as Sales_1,
    #     CASE WHEN sum(Day_14_total_sales) = 0 THEN 0 ELSE sum(Spend)/sum(Day_14_total_sales) END as ACOS_1 from T1
    #     where date_ = CURRENT_DATE - INTERVAL '1 day'
    #     group by portfolio_name
    #     )
    #     select T2_MTD.portfolio_name, Spend_MTD, Sales_MTD, ACOS_MTD, Spend_1, Sales_1, ACOS_1 from T2_MTD
    #     left join T3_1 on T2_MTD.portfolio_name = T3_1.portfolio_name
    #     """,conn_post_2)
    # df_ACOS_Spend.columns = df_ACOS_Spend.columns.str.upper()
    # df_Por_Cat = pd.read_sql_query(f"""
    #     with t1 as (
    #     select distinct portfolio_name, advertised_asin from public.ad_products
    #         where amz_account in ('A1AP6U5UUAP9DL','A2PXP9SSI7GDVG','A2U7X22FDY3QZO','AVXCWFLFGENR','A1ITUVWQWSB9L0')
    #         and advertised_asin is not null
    #         and portfolio_name not in ('Test','Not grouped','-')
    #         and date_ >= CURRENT_DATE - INTERVAL '3 day'
    #     ), t2 as (
    #     Select portfolio_name, Category from t1
    #     left join
    #     (select asin, Category from mi_report.launching_plan) t3
    #     on t1.advertised_asin = t3.asin
    #     )
    #     select distinct portfolio_name, Category from t2
    #     where Category is not null
    # """,conn_post_2)
    # df_Por_Cat.columns = df_Por_Cat.columns.str.upper()    
#### Tạo bảng ACOS - PPC - MTD
    # df_Spend = df_ACOS_Spend.merge(df_Por_Cat, on ='PORTFOLIO_NAME', how='left')
    
    # def categorize_row(row):
    #     if row['CATEGORY'] == 'Furniture - Indoor' and row['PORTFOLIO_NAME'] == '[Indoor Furniture] Papasan Chair':
    #         return 'Papasan Chair'
    #     elif row['CATEGORY'] == 'Furniture - Indoor' and row['PORTFOLIO_NAME'] != '[Indoor Furniture] Papasan Chair':
    #         return 'Furniture - Indoor (excl PPS)'
    #     else:
    #         return row['CATEGORY']
    # # Áp dụng hàm để tạo cột mới 'CATE OKR'
    # df_Spend['CATE OKR'] = df_Spend.apply(categorize_row, axis=1)
    # Spend_MTD = df_Spend.groupby('CATE OKR').agg({'SPEND_MTD':'sum','SALES_MTD':'sum'}).reset_index()
    # Spend_MTD.loc[len(Spend_MTD)] = ['Gardening + Outdoor Entertainment',
    #                                  Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Gardening', 'SPEND_MTD'].values[0]
    #                                  + Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Outdoor Entertainment', 'SPEND_MTD'].values[0],
    #                                  Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Gardening', 'SALES_MTD'].values[0]
    #                                  + Spend_MTD.loc[Spend_MTD['CATE OKR'] == 'Outdoor Entertainment', 'SALES_MTD'].values[0],]
    # Spend_MTD_1 = Spend_MTD[~Spend_MTD['CATE OKR'].isin(['Gardening', 'Outdoor Entertainment'])]
    # Spend_MTD_1['ACOS MTD'] = Spend_MTD_1['SPEND_MTD']/Spend_MTD_1['SALES_MTD']
    # Spend_MTD_2 = Spend_MTD_1.reset_index(drop=True)
    # Spend_MTD_2.loc[len(Spend_MTD_2)] = ['', Spend_MTD_2['SPEND_MTD'].sum(),Spend_MTD_2['SALES_MTD'].sum(),
    #                                          Spend_MTD_2['SPEND_MTD'].sum()/Spend_MTD_2['SALES_MTD'].sum(),]
    # Spend_MTD_3 = Spend_MTD_2.rename(columns={'SPEND_MTD': 'SPEND PPC MTD'})
#### Tạo bảng MTD sell out và Sell out
    df_MTD = pd.concat([df_real_sales_MTD, df_real_time_sales_MTD])
    df_MTD_sum = df_MTD.groupby(['ASIN']).agg({'SELLOUTMTD': 'sum','GMVOUTMTD': 'sum'}).reset_index()
    #df_sell_out = df_MTD_sum.merge(df_sell_out_today, left_on ='ASIN', right_on= 'ASIN', how='left')   
#### Table 1
#### df_in_out = df_sell_out.merge(df_sell_in, left_on ='ASIN', right_on= 'Asin', how='left')
    df_sell_out_info = df_info.merge(df_MTD_sum, left_on ='ASIN', right_on= 'ASIN', how='left')
    df_sell_out_info_0 = df_sell_out_info.merge(df_sell_out_today, left_on ='ASIN', right_on= 'ASIN', how='left')
    df_sell_out_info_1 = df_sell_out_info_0.merge(df_WM, left_on ='SKU', right_on= 'SKU', how='left')
    df_sell_out_info_2 = df_sell_out_info_1.merge(df_WF, left_on ='SKU', right_on= 'SKU', how='left')
    df_sell_out_info_3 = df_sell_out_info_2.merge(df_AMZ_seller, left_on ='SKU', right_on= 'SKU', how='left')
    df_sell_out_info_4 = df_sell_out_info_3.fillna(0)
    df_sell_out_info_4['Total Orders Today'] = df_sell_out_info_4.apply(lambda row: row['SELLOUT'] + row['SELLWM'] + row['SELLWF'] + row['SELLASC'], axis=1)
    df_sell_out_info_4['Total GMV Today'] = df_sell_out_info_4.apply(lambda row: row['ORDERED_REVENUE'] + row['GMV_WM'] + row['GMV_WF'] + row['GMV_ASC'], axis=1)
    df_sell_out_info_4['Total Orders MTD'] = df_sell_out_info_4.apply(lambda row: row['SELLOUTMTD'] + row['WM_MTD'] + row['WF_MTD'] + row['ASC_MTD'], axis=1)
    df_sell_out_info_4['Total GMV MTD'] = df_sell_out_info_4.apply(lambda row: row['GMVOUTMTD'] + row['GMV_WM_MTD'] + row['GMV_WF_MTD'] + row['GMV_ASC_MTD'], axis=1)
#### data_summary = df_sell_out_info_3.merge(df_sell_in, left_on ='ASIN', right_on= 'Asin', how='left')
    data_summary_1 = df_sell_out_info_4.groupby(['Sales PIC','Category', 'Product Group']).agg(
        {'SELLOUT': 'sum', 'ORDERED_REVENUE': 'sum', 'SELLOUTMTD': 'sum', 'GMVOUTMTD': 'sum', 
         'SELLWM': 'sum', 'GMV_WM': 'sum', 'WM_MTD': 'sum', 'GMV_WM_MTD': 'sum',
         'SELLWF': 'sum', 'GMV_WF':'sum', 'WF_MTD': 'sum', 'GMV_WF_MTD': 'sum', 
         'SELLASC': 'sum', 'ASC_MTD': 'sum', 'GMV_ASC':'sum', 'GMV_ASC_MTD': 'sum',
         'Total Orders Today': 'sum', 'Total GMV Today': 'sum', 'Total Orders MTD': 'sum', 'Total GMV MTD': 'sum', 
        }).reset_index()
    data_summary_2 = data_summary_1.rename(columns=
        {'SELLOUT': 'AMZ AVC','ORDERED_REVENUE': 'GMV AMZ AVC', 'SELLOUTMTD': 'AMZ AVC MTD', 'GMVOUTMTD': 'GMV AMZ AVC MTD',
         'SELLWM': 'Walmart','GMV_WM': 'GMV Walmart', 'WM_MTD': 'Walmart MTD', 'GMV_WM_MTD': 'GMV Walmart MTD',
         'SELLWF': 'Wayfair','GMV_WF':'GMV Wayfair', 'WF_MTD': 'Wayfair MTD', 'GMV_WF_MTD': 'GMV Wayfair MTD',
         'SELLASC': 'AMZ ASC','GMV_ASC':'GMV AMZ ASC', 'ASC_MTD': 'AMZ ASC MTD', 'GMV_ASC_MTD': 'GMV AMZ ASC MTD',
        })
    data_summary_3 = data_summary_2[['Sales PIC','Category','Product Group',
                                     'Total Orders MTD', 'AMZ AVC MTD', 'AMZ ASC MTD', 'Walmart MTD', 'Wayfair MTD',
                                     'Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                                     'Total Orders Today', 'AMZ AVC', 'AMZ ASC', 'Walmart', 'Wayfair',
                                     'Total GMV Today', 'GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']]
    data_summary_4 = data_summary_3[(data_summary_3['Total Orders MTD'] != 0) | (data_summary_3['Total Orders Today'] != 0)]
    data_summary_4 = data_summary_4.sort_values(['Category','Sales PIC'],ascending=[True,True])    
# #### Table1.2
#     data_summary_5 = data_summary_4[~((data_summary_4['Category'].str.contains('Services')))] 
# #### Table1.3
#     data_summary_Services = data_summary_4[(data_summary_4['Category'].str.contains('Services'))]
    def Row_Total(data): 
        data_1 = data.reset_index(drop=True)
        data_1.loc[len(data_1)] = ['','', 'Total', data_1['Total Orders MTD'].sum(), data_1['AMZ AVC MTD'].sum(),
                                               data_1['AMZ ASC MTD'].sum(), data_1['Walmart MTD'].sum(),
                                               data_1['Wayfair MTD'].sum(), data_1['Total GMV MTD'].sum(),
                                               data_1['GMV AMZ AVC MTD'].sum(), data_1['GMV AMZ ASC MTD'].sum(),
                                               data_1['GMV Walmart MTD'].sum(), data_1['GMV Wayfair MTD'].sum(),
                                               data_1['Total Orders Today'].sum(), data_1['AMZ AVC'].sum(),
                                               data_1['AMZ ASC'].sum(), data_1['Walmart'].sum(),
                                               data_1['Wayfair'].sum(), data_1['Total GMV Today'].sum(),
                                               data_1['GMV AMZ AVC'].sum(), data_1['GMV AMZ ASC'].sum(),
                                               data_1['GMV Walmart'].sum(), data_1['GMV Wayfair'].sum(),]

        data_1 = data_1.reindex([len(data_1) - 1] + list(range(len(data_1) - 1)))
        data_1[['Total Orders MTD','AMZ AVC MTD','AMZ ASC MTD','Walmart MTD','Wayfair MTD',
                    'Total Orders Today','AMZ AVC','AMZ ASC','Walmart','Wayfair']] = data_1[
                    ['Total Orders MTD','AMZ AVC MTD','AMZ ASC MTD','Walmart MTD','Wayfair MTD',
                    'Total Orders Today','AMZ AVC','AMZ ASC','Walmart','Wayfair']].astype(int).applymap(format_number)
        data_1[['Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                   'Total GMV Today','GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']] = data_1[
                    ['Total GMV MTD','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                   'Total GMV Today','GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']].astype(int).applymap(format_number_1)
        return data_1   
    data_summary_5 = Row_Total(data_summary_4)
#     data_summary_Services_1 = Row_Total(data_summary_Services)        
#### Table1.1
    df_okr_cat = data_summary_4[['Category','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV Walmart MTD','GMV Wayfair MTD',
                                'GMV AMZ AVC','GMV AMZ ASC','GMV Walmart','GMV Wayfair']]
    df_okr_cat_1 = df_okr_cat.groupby(['Category']).agg(
        {'GMV AMZ AVC MTD': 'sum', 'GMV AMZ ASC MTD': 'sum', 'GMV Walmart MTD': 'sum', 'GMV Wayfair MTD': 'sum', 
         'GMV AMZ AVC': 'sum', 'GMV AMZ ASC': 'sum', 'GMV Walmart': 'sum', 'GMV Wayfair': 'sum',
        }).reset_index()
    df_okr_cat_AMZ_1 = df_okr_cat_1[['Category','GMV AMZ AVC MTD','GMV AMZ ASC MTD','GMV AMZ AVC','GMV AMZ ASC']]
    df_okr_cat_AMZ_1['GMV MTD'] = df_okr_cat_AMZ_1.apply(lambda row: row['GMV AMZ AVC MTD'] + row['GMV AMZ ASC MTD'] , axis=1)
    df_okr_cat_AMZ_1['GMV Today'] = df_okr_cat_AMZ_1.apply(lambda row: row['GMV AMZ AVC'] + row['GMV AMZ ASC'] , axis=1)
    df_okr_cat_AMZ_2 = df_okr_cat_AMZ_1[['Category','GMV MTD','GMV Today']]
    df_okr_cat_AMZ_2.loc[len(df_okr_cat_AMZ_2)] = ['Gardening + Outdoor Entertainment',
                        df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Gardening', 'GMV MTD'].values[0]
                        + df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Outdoor Entertainment', 'GMV MTD'].values[0],
                        df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Gardening', 'GMV Today'].values[0]
                        + df_okr_cat_AMZ_2.loc[df_okr_cat_AMZ_2['Category'] == 'Outdoor Entertainment', 'GMV Today'].values[0]]
    df_okr_cat_AMZ_3 = df_okr_cat_AMZ_2[~df_okr_cat_AMZ_2['Category'].isin(['Gardening', 'Outdoor Entertainment'])]
    df_okr_cat_WM_1 = df_okr_cat_1[['Category','GMV Walmart MTD','GMV Walmart']]
    df_okr_cat_WM_1.loc[len(df_okr_cat_WM_1)] = ['Walmart',
                                                 df_okr_cat_WM_1['GMV Walmart MTD'].sum(),df_okr_cat_WM_1['GMV Walmart'].sum()]
    df_okr_cat_WM_2 = df_okr_cat_WM_1[df_okr_cat_WM_1['Category']=='Walmart']
    df_okr_cat_WM_3 = df_okr_cat_WM_2.rename(columns={'GMV Walmart MTD': 'GMV MTD','GMV Walmart': 'GMV Today', })
    df_okr_cat_WF_1 = df_okr_cat_1[['Category','GMV Wayfair MTD','GMV Wayfair']]
    df_okr_cat_WF_1.loc[len(df_okr_cat_WF_1)] = ['Wayfair',
                                                 df_okr_cat_WF_1['GMV Wayfair MTD'].sum(), df_okr_cat_WF_1['GMV Wayfair'].sum(),]
    df_okr_cat_WF_2 = df_okr_cat_WF_1[df_okr_cat_WF_1['Category']=='Wayfair']
    df_okr_cat_WF_3 = df_okr_cat_WF_2.rename(columns={'GMV Wayfair MTD': 'GMV MTD','GMV Wayfair': 'GMV Today', })
    df_okr_cat_2 = pd.concat([df_okr_cat_AMZ_3, df_okr_cat_WM_3, df_okr_cat_WF_3], ignore_index=True)
    df_okr_cat_2['Category'] = df_okr_cat_2['Category'].apply(lambda x: 'Services' if 'Services' in x else x)
    df_okr_cat_3 = df_okr_cat_2.groupby(df_okr_cat_2['Category']).sum().reset_index()    
    df_okr_cat_4 = df_okr_cat_3.reset_index(drop=True)
    df_okr_cat_4.loc[len(df_okr_cat_4)] = ['', df_okr_cat_4['GMV MTD'].sum(), df_okr_cat_4['GMV Today'].sum(),]
    df_okr_cat_5 = df_OKR_GG.merge(df_okr_cat_4, left_on ='Category', right_on= 'Category', how='left')
#     df_okr_cat_5 = df_okr_cat_5.reindex([len(df_okr_cat_5) - 1] + list(range(len(df_okr_cat_5) - 1)))
    df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].replace('[^\d.]', '', regex=True)
    df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].replace(',', '', regex=True)
    df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']] = df_okr_cat_5[['Total Order OKR','OKR Today','OKR MTD']].astype(float)
    df_okr_cat_5['% GMV MTD / OKR MTD'] = df_okr_cat_5.apply(lambda row: row['GMV MTD'] / row['OKR MTD'] 
                                                               if row['Total Order OKR'] != 0 else "-", axis=1)
    df_okr_cat_5['% GMV Today / OKR Today'] = df_okr_cat_5.apply(lambda row: row['GMV Today'] / row['OKR Today']
                                                                 if row['OKR Today'] != 0 else "-", axis=1)    
#### Ghép với bảng ACOS - PPC - MTD
    # df_okr_cat_6 = df_okr_cat_5.merge(Spend_MTD_3, left_on ='Category', right_on = 'CATE OKR', how='left') # uncomment later
    df_okr_cat_6 = df_okr_cat_5 # comment later
    df_okr_cat_7 = df_okr_cat_6.fillna(0)
    df_okr_cat_8 = df_okr_cat_7[['Market Place','Category','Total Order OKR','GMV MTD','OKR MTD','% GMV MTD / OKR MTD',
                            # 'SPEND PPC MTD','ACOS MTD', # uncomment later
                            'GMV Today','OKR Today','% GMV Today / OKR Today']]
    df_okr_cat_8[['Total Order OKR','GMV MTD','OKR MTD','GMV Today','OKR Today'
                # ,'SPEND PPC MTD' # uncomment later
                  ]] = df_okr_cat_8[['Total Order OKR','GMV MTD','OKR MTD','GMV Today','OKR Today'
                #   ,'SPEND PPC MTD' # uncomment later
                  ]].astype(int).applymap(format_number)
    df_okr_cat_8[['% GMV MTD / OKR MTD','% GMV Today / OKR Today'
                #   ,'ACOS MTD' # uncomment later
                  ]] = df_okr_cat_8[
    ['% GMV MTD / OKR MTD','% GMV Today / OKR Today'
    #  ,'ACOS MTD' # uncomment later
     ]].applymap(lambda x: format_gap(x) if x != "-" else x)
    df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'Category'] = ' '
    df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'Category'] = ' '
    # df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'SPEND PPC MTD'] = 'No data' # uncomment later
    # df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'SPEND PPC MTD'] = 'No data'
    # df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Walmart', 'ACOS MTD'] = 'No data'
    # df_okr_cat_8.loc[df_okr_cat_8['Market Place'] == 'Wayfair', 'ACOS MTD'] = 'No data'
    #Hard Code Off-site
    df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'GMV MTD'] = 'No data'
    df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', '% GMV MTD / OKR MTD'] = 'No data'
    # df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'SPEND PPC MTD'] = 'No data' # uncomment later
    # df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'ACOS MTD'] = 'No data'
    # df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', 'GMV Today'] = 'No data'
    # df_okr_cat_8.loc[df_okr_cat_8['Category'] == 'Off-site', '% GMV Today / OKR Today'] = 'No data'

    df_okr_cat_9 = df_okr_cat_8[df_okr_cat_8['Market Place']!='']    
#### Get Target MTD and Daily 
    Table_ORK = df_OKR_GG[df_OKR_GG['Category'] == 'Total']
    Table_ORK_1 = Table_ORK.reset_index(drop=True)
    OKR_OCT = Table_ORK_1['Total Order OKR'].replace('[^\d.]', '', regex=True).astype(float)
    OKR_Target = Table_ORK_1['OKR Team'].replace('[^\d.]', '', regex=True).astype(float)
    OKR_Target_Base = Table_ORK_1['OKR Base'].replace('[^\d.]', '', regex=True).astype(float)
    OKR_Target_MTD = Table_ORK_1['OKR MTD'].replace('[^\d.]', '', regex=True).astype(float)    
#### Alert OKR
    # Lấy Time Gone
    day_of_month = data_date.day
    month = data_date.month
    year = data_date.year
    # Sử dụng module calendar để lấy số ngày trong tháng
    _, num_days = calendar.monthrange(year, month)
    Time_gone = day_of_month/num_days    
    GMV_today = data_summary_4['Total GMV Today'].sum()
    GMV_MTD = data_summary_4['Total GMV MTD'].sum()
    Target_OKR = format_number_1(OKR_OCT[0])    
    if GMV_MTD > OKR_Target_MTD[0]:
        OKR_Alert = OKR_Target_Base[0]
        Gap = GMV_today / OKR_Target_Base[0]
    else:
        OKR_Alert = OKR_Target[0]
        Gap = GMV_today / OKR_Target[0]   
    Gap_MTD = GMV_MTD / OKR_OCT[0]
    Gap_per = format_gap(Gap)
    Gap_MTD_per = format_gap(Gap_MTD)
    Time_gone_per = format_gap(Time_gone)     
#### Table 2
    data_detail_sell_in = df_detail_sell_in.merge(df_info, left_on ='ASIN', right_on= 'ASIN', how='left')
    data_detail_sell_in_1 = data_detail_sell_in.groupby(['Sales PIC','Category', 'Product Group']).agg(
        {'AVCDS': 'sum', 'AVCWH': 'sum', 'AVCDI': 'sum','ALLCHANNEL': 'sum'}).reset_index()
    data_detail_sell_in_2 = data_detail_sell_in_1.rename(columns=
        {'AVCDS': 'AVC DS', 'AVCWH': 'AVC WH','AVCDI': 'AVC DI','ALLCHANNEL': 'All Channel'})
    data_detail_sell_in_2 = data_detail_sell_in_2.sort_values(['Category','Sales PIC'],ascending=[True,True])
    data_detail_sell_in_2.loc[len(data_detail_sell_in_2)] = ['','', 'Tổng', data_detail_sell_in_2['AVC DS'].sum(), 
    data_detail_sell_in_2['AVC WH'].sum(),data_detail_sell_in_2['AVC DI'].sum(),data_detail_sell_in_2['All Channel'].sum()]
    data_detail_sell_in_2[['AVC DS','AVC WH','AVC DI','All Channel']] = data_detail_sell_in_2[
        ['AVC DS','AVC WH','AVC DI','All Channel']].astype(int).applymap(format_number)
    data_detail_sell_in_3 = data_detail_sell_in_2[data_detail_sell_in_2['Category']!="#N/A"]
    data_detail_sell_in_4 = data_detail_sell_in_3[data_detail_sell_in_3['All Channel']!="0"]   
##### Điều kiện 1: SellOut today > 50% SellOut Avg 30day
    sell_out_today = data_sell_out_today.iloc[0]['sellouttoday']
    sell_out_avg30d = data_sell_out_avg30d.iloc[0]['avg30daysellout']
#    return print(data_summary_2,data_detail_sell_in_2)   
#### Điều kiện 2: Check Data ACOS và PPC MTD
    # Check_ACOS = df_ACOS_Spend[['SPEND_1']].dropna().sum()    # uncomment later
#### Đổi màu tên cột
    def format_name_color(table, col):
        first_row_t1 = table.find_all('tr')[1]
        cell = first_row_t1.find_all('th')[col]
        cell['style'] = cell['style'].replace('#548235', '#305496')
        return table
#### Đổi màu tên cột services
    def format_name_color_services(table, col):
        first_row_t1 = table.find_all('tr')[0]
        cell = first_row_t1.find_all('th')[col]
        cell['style'] = cell['style'].replace('#548235', '#305496')
        return table       
#### Đổi màu cột
    def format_color(table, col):
        table_1 = table.select(f'td:nth-child({col})')
        for cell in table_1:
            cell['style'] = cell['style'].replace('#E2EFDA', '#D9E1F2')
        return table
#### Căn các dòng bằng nhau (70px)
    def format_width(table, col):
        table_1 = table.select(f'td:nth-child({col})')
        for cell in table_1:
            cell['style'] = cell['style'] + ';width: 70px'
        return table        
#### Conditional formatting %
    def conditional_formatting(table, col_data_4, col_data_5, Time_gone):
        # Select the data cells in the specified columns for the first row
        data_cells_4_row1 = table.select(f'tr:nth-child(1) td:nth-child({col_data_4})')
        # Apply background color to the first row based on the condition for column 4
        for cell in data_cells_4_row1:
            text = cell.text.strip()
            value = re.findall(r'\d+\.\d+|\d+', text)
            if value and float(value[0]) < Time_gone*100:
                cell['style'] += ';' + 'background-color: #e06666'
            elif value and float(value[0]) >= Time_gone*100:
                cell['style'] += ';' + 'background-color: #b7e1cd'
            else:
                pass
        # Select the data cells in the specified column for rows 2 and onward
        data_cells_5_remaining_rows = table.select(f'tr:nth-child(n+2) td:nth-child({col_data_5})')
        for cell in data_cells_5_remaining_rows:
            text = cell.text.strip()
            value = re.findall(r'\d+\.\d+|\d+', text)
            if value and float(value[0]) < Time_gone*100:
                cell['style'] += ';' + 'background-color: #e06666'
            elif value and float(value[0]) >= Time_gone*100:
                cell['style'] += ';' + 'background-color: #b7e1cd'
            else:
                pass          
        return table
#### Conditional formatting ACOS    
    def conditional_formatting_ACOS(table, col):
        table_1 = table.select(f'td:nth-child({col})')
        for cell in table_1:
            text = cell.text.strip()  # Xóa khoảng trắng ở đầu và cuối chuỗi
            value = re.findall(r'\d+\.\d+|\d+', text)  # Trích xuất số từ chuỗi
            if value and float(value[0]) <= 20:
                cell['style'] += ';' + 'background-color: #b7e1cd'
            elif value and float(value[0]) <= 40:
                cell['style'] += ';' + 'background-color: #fff2cc'
            elif value and float(value[0]) <= 70:
                cell['style'] += ';' + 'background-color: #f4cccc' 
            elif value and float(value[0]) > 70 :
                cell['style'] += ';' + 'background-color: #e06666'
            elif text == 'Zero Sales' or text == 'Zero Sales Ads':
                cell['style'] += ';' + 'background-color: #e06666'
            elif text == 'NLY' or text == 'No Spent' or text == 'NA' or text == 'No data':
                cell['style'] += ';' + 'background-color: #b7b7b7'
            else:
                pass
        table_2 = str(table)
        return table
    if sell_out_today <= sell_out_avg30d*0:
        # Tổng hợp thông tin nếu có lỗi
        print('Data Real Time Sales Có lỗi')
        print(f'Tổng sales out today: {sell_out_today}')
        print(f'80% Tổng số sales out trong 30 ngày gần nhất: {sell_out_avg30d*0.5}')
        data_summary_2
        data_detail_sell_in_2
        sys.exit()
    # elif Check_ACOS[0] == 1000:     # uncomment later
    #     print('Chưa có data SEM')    
    else:      
######## Table Category
#         header_col_table_category = list(df_okr_cat_6.columns.values)
#         col_table_category = ['','','','','Time Gone',Time_gone_per,'','','']
#         mapping_table_category = zip(col_table_category, header_col_table_category)
#         col_list_table_category = pd.MultiIndex.from_tuples(mapping_table_category)
#         df_okr_cat_6.columns = col_list_table_category    
        table_category = build_table(df_okr_cat_9, 'green_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='3px 3px 3px 3px', width='auto',) 
#     #### Gộp cell Tổng            
#         replacements_cat = {
#         '<td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto">Total</td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>':
#         '<td colspan="2" halign="center" style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Total</td>', }
#         for old_value, new_value in replacements_cat.items():
#             table_category = table_category.replace(old_value, new_value)
        table_category_1 = BeautifulSoup(table_category, 'html.parser')  
#     #### Tô màu hàng Tổng
#         last_row_cat = table_category_1.find_all('tr')[1]
#         cells_last_row_cat = last_row_cat.find_all('td')
#         for cell in cells_last_row_cat:
#             cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto''' 
    #### Tô màu tên cột
        for i in (2,3,4,5,6,7):
            table_category_2 = format_name_color_services(table_category_1, i)  
    #### Tô màu cột
        for i in (3,4,5,6,7,8):
            table_category_2 = format_color(table_category_1, i)  
        #table_category_3 = conditional_formatting(table_category_2, 5,Time_gone)
        table_category_3 = conditional_formatting(table_category_2, 6, 6, 1)
        table_category_3 = conditional_formatting(table_category_2, 11, 11, 1)
        table_category_3 = conditional_formatting_ACOS(table_category_2, 8)
        table_category_4 = str(table_category_3)        
######## Table Product   
        header_col_table_product = list(data_summary_5.columns.values)
        col_table_product = ['','','','','','','','Total OKR',Target_OKR,Gap_MTD_per,'Time Gone',Time_gone_per,'',
       #'Total OKR',Target_OKR,'N/A','Time Gone','N/A','',
       '','','','',f'Target {formatted_time_daily}',format_number_1(OKR_Alert),Gap_per,'','','',]
        mapping_table_product = zip(col_table_product, header_col_table_product)
        col_list_table_product = pd.MultiIndex.from_tuples(mapping_table_product)
        data_summary_5.columns = col_list_table_product             
        table_product = build_table(data_summary_5, 'green_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='3px 3px 3px 3px', width='auto',)
    #### Gộp cell Tổng            
        replacements_pro = {
        '<td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto"></td>\n      <td style = "background-color: #E2EFDA;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 3px 3px 3px 3px;width: auto">Total</td>':
        '<td colspan="3" halign="center" style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Total</td>',        }
        for old_value, new_value in replacements_pro.items():
            table_product = table_product.replace(old_value, new_value)
        table_product_1 = BeautifulSoup(table_product, 'html.parser')  
    #### Tô màu hàng Tổng
        last_row_pro = table_product_1.find_all('tr')[2]
        cells_last_row_pro = last_row_pro.find_all('td')
        for cell in cells_last_row_pro:
            cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''              
    #### Tô màu Alert
        first_row_t1 = table_product_1.find_all('tr')[0]
        cells_first_row_t1_OKR = first_row_t1.find_all('th')[8]
        cells_first_row_t1_per_OKR = first_row_t1.find_all('th')[9]
        cells_first_row_t1_gap_OKR = first_row_t1.find_all('th')[5]
        cells_first_row_t1_gap_Time_gone = first_row_t1.find_all('th')[3]
        cells_first_row_t1_OKR['style'] = cells_first_row_t1_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
        cells_first_row_t1_per_OKR['style'] = cells_first_row_t1_per_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
        cells_first_row_t1_gap_OKR['style'] = cells_first_row_t1_gap_OKR['style'].replace('border-bottom: 2px solid #548235;', '') 
        cells_first_row_t1_gap_Time_gone['style'] = cells_first_row_t1_gap_Time_gone['style'].replace('border-bottom: 2px solid #548235;', '') 
        if GMV_today >= OKR_Alert:
            cells_first_row_t1_OKR['style'] += ';' + 'background-color: #0a6b3d'
            cells_first_row_t1_per_OKR['style']+= ';' + 'background-color: #0a6b3d'
        else:
            cells_first_row_t1_OKR['style'] += ';' + 'background-color: #e06666'
            cells_first_row_t1_per_OKR['style'] += ';' + 'background-color: #e06666'

        if Gap_MTD >= Time_gone:
            cells_first_row_t1_gap_OKR['style'] += ';' + 'background-color: #0a6b3d'
            cells_first_row_t1_gap_Time_gone['style'] += ';' + 'background-color: #0a6b3d'
        else:
            cells_first_row_t1_gap_OKR['style'] += ';' + 'background-color: #e06666'
            cells_first_row_t1_gap_Time_gone['style'] += ';' + 'background-color: #e06666'            
    #### Tô màu cột
        for i in (3,4,5,6,7,13,14,15,16,17):
            table_product_2 = format_name_color(table_product_1, i)  
        for i in (4,5,6,7,8,14,15,16,17,18):
            table_product_2 = format_color(table_product_1, i)    
        for i in range(4,24):
            table_product_2 = format_width(table_product_1, i)  
        table_product_3 = str(table_product_2)         
######## Table2 = data_detail_sell_in_2.to_html(index=False)
        table2 = build_table(data_detail_sell_in_4,'orange_dark', font_size='11px', font_family='Arial, sans-serif',text_align='center', padding ='4px 4px 4px 4px')   
        if data_detail_sell_in_4.empty: 
            table2_2 = '<b> Không Có Data Sell In </b>'
        else:
            table2_1 = BeautifulSoup(table2, 'html.parser')  
        #### Tô màu hàng cuối
            last_row_t2 = table2_1.find_all('tr')[-1]
            cells_last_row_t2 = last_row_t2.find_all('td')
            for cell in cells_last_row_t2:
                cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''
            table2_2 = str(table2_1)    
######### Thiết lập thông tin tài khoản email
        # email = 'hieptn@yes4all.com'
        # password = 'bqaw gtob iobs zpal'
        # email = 'giangdt@yes4all.com'
        # password = 'fhvs lwgf pecd ljau'      

        email = 'antv@yes4all.com'
        password = 'kcus xmvl pugm orpe'        
######### Thiết lập đối tượng email
        msg = MIMEMultipart()       
        # recipients = ['deploymentlab@yes4all.com','thuyle@yes4all.com', 'hungvo@yes4all.com','dinhnd@yes4all.com',
        #               'tranghh@yes4all.com','tuanta@yes4all.com','tienttt@yes4all.com','ngocntm@yes4all.com',
        #               'vylm@yes4all.com','leader_innonet@yes4all.com', 'yennth@yes4all.com']
        # cc_recipients = ['thuyle@yes4all.com', 'hungvo@yes4all.com','dinhnd@yes4all.com','tranghh@yes4all.com',
        #                  'tuanta@yes4all.com','tienttt@yes4all.com','ngocntm@yes4all.com','vylm@yes4all.com', 'antv@yes4all.com', 
        #                 'leader_innonet@yes4all.com']
        # msg['To'] = 'deploymentlab@yes4all.com'
        # msg['CC'] = ", ".join(cc_recipients)

        # recipients = 'giangdt@yes4all.com'
        # msg['To'] = 'giangdt@yes4all.com' 
        recipients = 'antv@yes4all.com'
        msg['To'] = 'antv@yes4all.com'   

        msg['From'] = email 
        msg['Subject'] = f'DAILY SALES PERFORMANCE UPDATE - {Subject_time}'        
######### Tạo nội dung HTML
        html = f"""
        <html>
            <body>
                <p>Dear team,</p>
                <p>Em xin update mail <b>Daily Performance</b> của team mình <b>{formatted_time}</b> cho 
                các project NPD và SLOB như sau:</p>
                <p>Version OKR : 16/01/2024</p>
                <h3><font color="Blue"><b>A. SELL OUT </b></font></h3> 
                <h4><font color="Blue"><b>I. CATEGORY </b></font></h4> 
                    {table_category_4}
                <h4><font color="Blue"><b>II. PRODUCT GROUP </b></font></h4> 
                <p>AMZ AVC: Amazon Vender Central<br>
                AMZ ASC: Amazon Seller Central</p>
                    {table_product_3} 
                <h3><font color="Blue"><b>B. SALE IN </b></font></h3>
                    {table2_2}     
                <p>Cảm ơn Team.</p>
                   
                <p>Best Regards,<br>
                Đỗ Thu Giang<br>
                DA & P.MKT AM<br>
                Phone number: 0914 051 097<br>
                Email:        giangdt@yes4all.com<br></p>             
            </body>
        </html>
        """
######### Chuyển đổi nội dung HTML thành MIMEText
        text = MIMEText(html, 'html')
        msg.attach(text)
######### Thiết lập kết nối SMTP và gửi email
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(email, password)
        server.sendmail(email, recipients, msg.as_string())
        server.quit()
########sys.exit()
        print("Job executed at", datetime.datetime.now())
        print('Đã gửi mail daily')

### Run hàm

In [4]:
schedule.every().day.at("17:00").do(Auto_mail_daily)

while True:
    schedule.run_pending()
    time.sleep(10)

data time: 2024-02-22
Job executed at 2024-02-23 17:01:01.003401
Đã gửi mail daily
data time: 2024-02-23
Job executed at 2024-02-24 17:01:02.838378
Đã gửi mail daily
data time: 2024-02-24
Job executed at 2024-02-25 17:00:58.455733
Đã gửi mail daily


In [5]:
Auto_mail_daily()

data time: 2024-02-26
Job executed at 2024-02-27 10:59:17.587011
Đã gửi mail daily


### Alert OKR

In [ ]:
# GMV_MTD = df_sell_out_info_3['Total GMV MTD'].sum()
# GMV_today = df_sell_out_info_3['Total GMV'].sum()
# OKR_OCT = df_OKR['OKR_TOTAL'].sum()

# # Ngày đầu tháng, cuối tháng
# first_day_of_month = data_date.replace(day=1)
# last_day_of_month = data_date + pd.offsets.MonthEnd()

# # Tạo danh sách các ngày trong tháng
# date_list = [first_day_of_month.date() + timedelta(days=x) 
#              for x in range(0, (last_day_of_month.date() - first_day_of_month.date()).days + 1)]

# # Chuyển danh sách thành DataFrame
# df_month = pd.DataFrame(date_list, columns=["Date"])

# # Chuyển đổi cột 'Date' thành đối tượng datetime
# df_month['Date'] = pd.to_datetime(df_month['Date'])

# # Lấy số ngày trong tháng cho mỗi ngày trong cột 'Date'
# df_month['Days_in_Month'] = df_month['Date'].dt.days_in_month

# # Tính số ngày còn lại trong tháng bằng cách trừ số ngày trong tháng cho ngày trong cột 'Date'
# df_month['Remaining_Days'] = df_month['Days_in_Month'] - df_month['Date'].dt.day + 1

# # Tạo một hàm ánh xạ tên ngày sang số
# def map_day_to_number(day_name):
#     days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
#     return days.index(day_name) + 1

# # Thêm cột 'Thứ' (ngày trong tuần)
# df_month['DOW'] = df_month['Date'].dt.day_name().map(map_day_to_number)

# # Lấy Weight Day
# df_wieght_days = pd.read_sql(f'''
#     with T1 as (
#     select sum(ordered_gmv) as ordered_gmv, DAYOFWEEK(log_date) AS day_of_week 
#     from public_main.sale_performance_by_channel_innonet
#     where ASIN in {list_asin}
#     and country = 'USA'
#     and label_datetime = 'daily'
#     and log_date <= '2023-09-28'
#     and log_date >=  '2023-09-01'
#     group by day_of_week 
#     )
#     select day_of_week, ordered_gmv, round(ordered_gmv/sum(ordered_gmv) OVER () * 100,2) AS Percentage, 
#     ordered_gmv/sum(ordered_gmv) OVER () - 1/7 AS Weihgt
#     from T1
#     group by day_of_week
#     ''', con=conn)

# # Left join lấy Weight
# df_month_1 = df_month.merge(df_wieght_days, left_on ='DOW', right_on= 'day_of_week', how='left')

# # Cumsum new Weight
# df_month_1['New_Weight'] = df_month_1['Weihgt'].iloc[::-1].cumsum()[::-1]

# # Chuyển đổi cột 'Date' thành đối tượng datetime
# df_month_1['Date'] = pd.to_datetime(df_month_1['Date']).dt.date

# # Lọc DataFrame bằng biến filter_date
# filtered_df = df_month_1[df_month_1['Date'] == data_date.date()]
# filtered_df_1 = filtered_df.reset_index()

In [ ]:
# filtered_df_1

In [ ]:
# GMV_daily = (OKR_OCT - (GMV_MTD - GMV_today)) / (filtered_df_1['Remaining_Days'][0] + filtered_df_1['New_Weight'][0])

# Lấy Time Gone
# day_of_month = data_date.day
# month = data_date.month
# year = data_date.year

# # Sử dụng module calendar để lấy số ngày trong tháng
# _, num_days = calendar.monthrange(year, month)

# Time_gone = day_of_month/num_days
# GMV_today = df_sell_out_info_4['Total GMV Today'].sum()
# GMV_MTD = df_sell_out_info_4['Total GMV MTD'].sum()
# #OKR_OCT = df_OKR['OKR_TOTAL'].sum()
# OKR_OCT = 2352537
# Target_OKR = format_number_1(OKR_OCT)

# OKR_Alert = 60979 #GMV_daily + GMV_daily*filtered_df_1['Weihgt'][0]

# OKR_Target = '$60,979'#format_number_1(OKR_Alert)

# Gap = GMV_today / OKR_Alert

# Gap_MTD = GMV_MTD / OKR_OCT

# def format_gap(x):
#     return "{:.2%}".format(x)

# Gap_per = format_gap(Gap)
# Gap_MTD_per = format_gap(Gap_MTD)
# Time_gone_per = format_gap(Time_gone)

## Table 3

In [ ]:
# # Table 3
# # Lấy số ngày trong tháng
# year = data_date.year
# month = data_date.month
# count_month_day = (datetime.datetime(year, month + 1, 1) - datetime.datetime(year, month, 1)).days
# month_name = data_date.strftime("%b")

# # Lấy ngày của date
# count_day = data_date.day

# # Số ngày còn lại
# ngay_con_lai = count_month_day - count_day

# # Lấy data GMV
# data_gmv_mtd = df_sell_out_info_3[['Sales PIC','ASIN','SKU','Category','Product Group','Product Name',
#                                   'GMVOUTMTD','GMV_WM_MTD','GMV_WF_MTD','Total GMV MTD']]
# # Left join OKR
# data_gmv_mtd_1 = data_gmv_mtd.merge(df_OKR, left_on ='SKU', right_on= 'SKU', how='left')

# data_gmv_mtd_2 = data_gmv_mtd_1.fillna(0)

# # Group by
# data_gmv_mtd_3 = data_gmv_mtd_2.groupby(['Sales PIC','Category', 'Product Group']).agg(
#     {'GMVOUTMTD': 'sum', 'GMV_WM_MTD': 'sum','GMV_WF_MTD':'sum','Total GMV MTD': 'sum',
#      'OKR_AMZ': 'sum', 'OKR_WM': 'sum','OKR_WF':'sum','OKR_TOTAL': 'sum',
#     }).reset_index()

# data_gmv_mtd_4 = data_gmv_mtd_3[~((data_gmv_mtd_3['Total GMV MTD'] == 0) & (data_gmv_mtd_3['OKR_TOTAL'] == 0))
#                                 & (data_gmv_mtd_3['Category'] != 'Services - FBA')
#                                ]

# data_gmv_mtd_4 = data_gmv_mtd_4.sort_values(['Category','Sales PIC'],ascending=[True,True])

# data_gmv_mtd_5 = data_gmv_mtd_4.reset_index(drop=True)

# data_gmv_mtd_5.loc[len(data_gmv_mtd_5)] = ['','', 'Tổng',
#                                            data_gmv_mtd_5['GMVOUTMTD'].sum(),
#                                            data_gmv_mtd_5['GMV_WM_MTD'].sum(),
#                                            data_gmv_mtd_5['GMV_WF_MTD'].sum(),
#                                            data_gmv_mtd_5['Total GMV MTD'].sum(),
#                                            data_gmv_mtd_5['OKR_AMZ'].sum(),
#                                            data_gmv_mtd_5['OKR_WM'].sum(),
#                                            data_gmv_mtd_5['OKR_WF'].sum(),
#                                            data_gmv_mtd_5['OKR_TOTAL'].sum()]

# # Di chuyển hàng mới lên đầu tiên bằng cách dùng df.reindex()
# data_gmv_mtd_5 = data_gmv_mtd_5.reindex([len(data_gmv_mtd_5) - 1] + list(range(len(data_gmv_mtd_5) - 1)))

# # Gộp AMZ và WM
# data_gmv_mtd_5['GMV AMZ+WM MTD'] = data_gmv_mtd_5.apply(lambda row: row['GMVOUTMTD'] + row['GMV_WM_MTD'], axis=1)
# data_gmv_mtd_5[f'OKR AMZ+WM {month_name}'] = data_gmv_mtd_5.apply(lambda row: row['OKR_AMZ'] + row['OKR_WM'], axis=1)

In [ ]:
# data_gmv_mtd_5

In [ ]:
# # Lấy các cột đạ gộp
# data_gmv_mtd_6 = data_gmv_mtd_5[['Sales PIC','Category','Product Group',
#                                  'GMV AMZ+WM MTD',f'OKR AMZ+WM {month_name}'
#                                  #'Total GMV MTD','GMV AMZ+WM MTD','GMV_WF_MTD',
#                                  #'OKR_TOTAL','OKR AMZ+WM MTD','OKR_WF'
#                                 ]]

# # OKR Daily
# #data_gmv_mtd_6['OKR Total MTD'] = data_gmv_mtd_6.apply(lambda row: row['OKR_TOTAL']/count_month_day*count_day, axis=1)
# data_gmv_mtd_6['OKR AMZ+WM MTD'] = data_gmv_mtd_6.apply(lambda row: row[f'OKR AMZ+WM {month_name}']/count_month_day*count_day, axis=1)
# #data_gmv_mtd_6['OKR WF MTD'] = data_gmv_mtd_6.apply(lambda row: row['OKR_WF']/count_month_day*count_day, axis=1)

# # % GAP OKR MTD
# def Precent_GAP(a, b):
#     if b == 0:
#         return "-"
#     elif a ==0:
#         return "-"
#     else:
#         return (a - b) / b

# #data_gmv_mtd_6['% Gap Total OKR MTD'] = data_gmv_mtd_6.apply(lambda row: 
# #                                                     Precent_GAP(row['Total GMV MTD'], row['OKR Total MTD']),axis=1)
# data_gmv_mtd_6['% Gap OKR AMZ+WM MTD'] = data_gmv_mtd_6.apply(lambda row: 
#                                                       Precent_GAP(row['GMV AMZ+WM MTD'], row['OKR AMZ+WM MTD']),axis=1)
# #data_gmv_mtd_6['% Gap OKR WF MTD'] = data_gmv_mtd_6.apply(lambda row: 
# #                                                 Precent_GAP(row['GMV_WF_MTD'], row['OKR WF MTD']), axis=1)

# #Avg GMV
# #data_gmv_mtd_6['Avg GMV Daily'] = data_gmv_mtd_6.apply(lambda row: row['Total GMV MTD']/count_day, axis=1)
# data_gmv_mtd_6['Avg GMV AMZ+WM Daily'] = data_gmv_mtd_6.apply(lambda row: row['GMV AMZ+WM MTD']/count_day, axis=1)
# #data_gmv_mtd_6['Avg GMV WF Daily'] = data_gmv_mtd_6.apply(lambda row: row['GMV_WF_MTD']/count_day, axis=1)

# #OKR Daily
# #data_gmv_mtd_6['Total OKR Daily'] = data_gmv_mtd_6.apply(lambda row: row['OKR_TOTAL']/count_month_day, axis=1)
# data_gmv_mtd_6['AMZ+WM OKR Daily'] = data_gmv_mtd_6.apply(lambda row: row[f'OKR AMZ+WM {month_name}']/count_month_day, axis=1)
# #data_gmv_mtd_6['WF OKR Daily'] = data_gmv_mtd_6.apply(lambda row: row['OKR_WF']/count_month_day, axis=1)

# #GAP OKR Daily
# # data_gmv_mtd_6['% Gap OKR Avg Daily'] = data_gmv_mtd_6.apply(lambda row: 
# #                                                      Precent_GAP(row['Avg GMV Daily'], row['Total OKR Daily']),axis=1)
# data_gmv_mtd_6['% Gap AMZ+WM OKR Avg Daily'] = data_gmv_mtd_6.apply(lambda row: 
#                                                       Precent_GAP(row['Avg GMV AMZ+WM Daily'], row['AMZ+WM OKR Daily']),axis=1)
# # data_gmv_mtd_6['% Gap WF OKR Avg Daily'] = data_gmv_mtd_6.apply(lambda row: 
# #                                                   Precent_GAP(row['Avg GMV WF Daily'], row['WF OKR Daily']), axis=1)

# #MTG
# #data_gmv_mtd_6['Total GMV MTG'] = data_gmv_mtd_6.apply(lambda row: row['OKR_TOTAL']-row['Total GMV MTD'], axis=1)
# data_gmv_mtd_6['AMZ+WM GMV MTG'] = data_gmv_mtd_6.apply(lambda row: row[f'OKR AMZ+WM {month_name}']-row['GMV AMZ+WM MTD'], axis=1)
# #data_gmv_mtd_6['WF GMV MTG'] = data_gmv_mtd_6.apply(lambda row: row['OKR_WF']-row['GMV_WF_MTD'], axis=1)

# #Daily to Go
# # data_gmv_mtd_6['Total Daily to go'] = data_gmv_mtd_6.apply(lambda row: row['Total GMV MTG']/ngay_con_lai, axis=1)
# data_gmv_mtd_6['AMZ+WM Daily to go'] = data_gmv_mtd_6.apply(lambda row: row['AMZ+WM GMV MTG']/ngay_con_lai, axis=1)
# # data_gmv_mtd_6['WF Daily to go'] = data_gmv_mtd_6.apply(lambda row: row['WF GMV MTG']/ngay_con_lai, axis=1)

# # #Daily to Go 95%
# # data_gmv_mtd_6['Total Daily to go 95%'] = data_gmv_mtd_6.apply(lambda row: row['Total GMV MTG']*0.95/ngay_con_lai, axis=1)
# # data_gmv_mtd_6['AMZ+WM Daily to go 95%'] = data_gmv_mtd_6.apply(lambda row: row['AMZ+WM GMV MTG']*0.95/ngay_con_lai, axis=1)
# # data_gmv_mtd_6['WF Dailt to go 95%'] = data_gmv_mtd_6.apply(lambda row: row['WF GMV MTG']*0.95/ngay_con_lai, axis=1)

In [ ]:
# data_gmv_mtd_6

In [ ]:
# # Bỏ các AMZ+WM GMV MTG > 0
# if data_gmv_mtd_6['AMZ+WM GMV MTG'].dtype == float or data_gmv_mtd_6['AMZ+WM GMV MTG'].dtype == int:
#     data_gmv_mtd_6.loc[data_gmv_mtd_6['AMZ+WM GMV MTG'] < 0, 'AMZ+WM GMV MTG'] = '-'
    
# if data_gmv_mtd_6['AMZ+WM Daily to go'].dtype == float or data_gmv_mtd_6['AMZ+WM Daily to go'].dtype == int:
#     data_gmv_mtd_6.loc[data_gmv_mtd_6['AMZ+WM Daily to go'] < 0, 'AMZ+WM Daily to go'] = '-'    

# # Đổi tên
# data_gmv_mtd_7 = data_gmv_mtd_6.rename(columns=
#     {'GMV_WF_MTD': 'GMV WF MTD',})

# # Lấy cột
# data_gmv_mtd_8 = data_gmv_mtd_7[['Sales PIC','Category','Product Group',
#                                  'GMV AMZ+WM MTD', f'OKR AMZ+WM {month_name}','OKR AMZ+WM MTD',
#                                  '% Gap OKR AMZ+WM MTD','Avg GMV AMZ+WM Daily','AMZ+WM OKR Daily',
#                                  'AMZ+WM GMV MTG','AMZ+WM Daily to go'
#                                  #Total GMV MTD','GMV WF MTD',
#                                  #% Gap Total OKR MTD','% Gap OKR WF MTD',
#                                  #Avg GMV Daily','Avg GMV WF Daily',
#                                  #'Total GMV MTG','WF GMV MTG',
#                                 ]]
# # Apply format $ 
# def format_currency(value):
#     if isinstance(value, str) and value == "-":
#         return value  
#     elif isinstance(value, (int, float)):
#         if value < 0:
#             formatted_value = '(${:,.0f})'.format(abs(value))
#         else:
#             formatted_value = '${:,.0f}'.format(value)
#         return formatted_value
#     return value

# data_gmv_mtd_8[[#'Total GMV MTD',
#                 'GMV AMZ+WM MTD',#'GMV WF MTD',
#                 f'OKR AMZ+WM {month_name}','OKR AMZ+WM MTD',
#                 #'Avg GMV Daily',
#                 'Avg GMV AMZ+WM Daily',#'Avg GMV WF Daily',
#                 'AMZ+WM OKR Daily',
#                 #'Total GMV MTG',
#                 'AMZ+WM GMV MTG',#'WF GMV MTG',
#                 'AMZ+WM Daily to go']] = data_gmv_mtd_8[
#                 [#'Total GMV MTD',
#                 'GMV AMZ+WM MTD',#'GMV WF MTD',
#                 f'OKR AMZ+WM {month_name}','OKR AMZ+WM MTD',
#                 #'Avg GMV Daily',
#                 'Avg GMV AMZ+WM Daily',#'Avg GMV WF Daily',
#                 'AMZ+WM OKR Daily',
#                 #'Total GMV MTG',
#                 'AMZ+WM GMV MTG',#'WF GMV MTG',
#                 'AMZ+WM Daily to go']].applymap(format_currency)

# # Apply format %
# def convert_decimal_to_percent(value):
#     if isinstance(value, (int, float)):
#         return f'{value * 100:.1f}%'
#     return value

# data_gmv_mtd_8[[#'% Gap Total OKR MTD',
#                 '% Gap OKR AMZ+WM MTD'#,'% Gap OKR WF MTD',
#                 #'% Gap OKR Avg Daily','% Gap AMZ+WM OKR Avg Daily','% Gap WF OKR Avg Daily'
#                 ]] = data_gmv_mtd_8[
#                 [#'% Gap Total OKR MTD',
#                 '% Gap OKR AMZ+WM MTD'#,'% Gap OKR WF MTD',
#                 #'% Gap OKR Avg Daily','% Gap AMZ+WM OKR Avg Daily','% Gap WF OKR Avg Daily'
#                 ]].applymap(convert_decimal_to_percent)

In [ ]:
# data_gmv_mtd_8

In [ ]:
# ######## Table 3
# ######## Chuyển đổi DataFrame thành chuỗi HTML       
# table3 = build_table(data_gmv_mtd_8, 'grey_dark', font_size='11px', font_family='Arial, sans-serif',
#                      text_align='center', padding ='3px 3px 3px 3px', width='auto',)

# def conditional_formatting_t3(table, col):
#     table_1 = table.select(f'td:nth-child({col})')

#     for cell in table_1:
#         text = cell.text.strip()  # Xóa khoảng trắng ở đầu và cuối chuỗi
#         value = re.findall(r'-?\d+\.\d+|\d+', text)  # Trích xuất số từ chuỗi
#         if value and float(value[0]) < 0:
#             cell['style'] = cell['style'] + ';background-color: #f4c7c3'#;color: black;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'
#         elif value and float(value[0]) > 0:
#             cell['style'] = cell['style'] + ';background-color: #b7e1cd'#;color: black;font-family: Arial, sans-serif;font-size: 11px;text-align: center;padding: 4px 4px 4px 4px;width: auto'
#         else:
#             pass

#     table_2 = str(table)

#     return table

In [ ]:
# table3_1 = BeautifulSoup(table3, 'html.parser')

# ######## Tô màu hàng Tổng
# last_row_t3 = table3_1.find_all('tr')[1]
# cells_last_row_t3 = last_row_t3.find_all('td')
# for cell in cells_last_row_t3:
#     cell['style'] = '''font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 12px;text-align: center;padding: 4px 4px 4px 4px;width: auto'''

# # for i in (7):
# #     table3_2 = conditional_formatting_t3(table3_1, 7)

# table3_2 = conditional_formatting_t3(table3_1, 7)

In [ ]:
# table3_3 = str(table3_2) 

In [ ]:
######## Gộp cell Tổng            
# replacements_3 = {
# '<td style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 12px;text-align: center;padding: 4px 4px 4px 4px;width: auto"></td>\n<td style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 12px;text-align: center;padding: 4px 4px 4px 4px;width: auto"></td>\n<td style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 12px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Tổng</td>':
# '<td colspan="3" halign="center" style="font-weight: bold; background-color: #CCCCCC;color: black;font-family: Arial, sans-serif; font-size: 12px;text-align: center;padding: 4px 4px 4px 4px;width: auto">Tổng</td>',
# }
# for old_value, new_value in replacements_3.items():
#     table3_3 = table3_3.replace(old_value, new_value)